[**Connect 4**](https://www.kaggle.com/competitions/connect-4) 

[**Connect X**](https://www.kaggle.com/competitions/connectx/leaderboard)  


[debugger_c4](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)


[agent_ConnectX-c4-1](https://www.kaggle.com/code/vyacheslavbolotin/agent-c4-1/edit/run/152769548)

[agent_ConnectX-c4-2](https://www.kaggle.com/code/vyacheslavbolotin/agent-c4-2/edit/run/152823182)

[agent_ConnectX-c4-3](https://www.kaggle.com/code/vyacheslavbolotin/agent-c4-3/edit/run/155271907)

[agent_ConnectX-c4-4](https://www.kaggle.com/code/vyacheslavbolotin/agent-c4-4/edit/run/155274900)

In [1]:
from kaggle_environments import make, evaluate

env__6_7   = make("connectx",configuration={"inarow":4,"columns": 7,"rows": 6})
env__10_11 = make("connectx",configuration={"inarow":5,"columns":11,"rows":10})

No pygame installed, ignoring import


In [2]:
env__10_11.run(["negamax", "negamax"])
env__10_11.render(mode="ipython", width=550, height=500)

## [agent-c4_1](https://www.kaggle.com/code/vyacheslavbolotin/agent-c4-1/edit/run/152769548)

In [3]:
def agent_c4_1(obs, config):

    WIDTH, N_STEPS = 2, 11

    map4q = '''
        30,31,32,33; 31,32,33,34; 32,33,34,35; 33,34,35,36; 
        40,41,42,43; 41,42,43,44; 42,43,44,45; 43,44,45,46; 
        50,51,52,53; 51,52,53,54; 52,53,54,55; 53,54,55,56; 
        04,14,24,34; 14,24,34,44; 24,34,44,54; 
        05,15,25,35; 15,25,35,45; 25,35,45,55; 
        06,16,26,36; 16,26,36,46; 26,36,46,56; 
        03,12,21,30; 
        04,13,22,31; 13,22,31,40; 
        05,14,23,32; 14,23,32,41; 23,32,41,50; 
        06,15,24,33; 15,24,33,42; 24,33,42,51; 
        16,25,34,43; 25,34,43,52; 
        26,35,44,53; 
        03,13,23,33; 13,23,33,43; 23,33,43,53; 
        20,31,42,53; 
        10,21,32,43; 21,32,43,54; 
        00,11,22,33; 11,22,33,44; 22,33,44,55; 
        01,12,23,34; 12,23,34,45; 23,34,45,56; 
        02,13,24,35; 13,24,35,46; 
        03,14,25,36; 
        00,10,20,30; 10,20,30,40; 20,30,40,50; 
        01,11,21,31; 11,21,31,41; 21,31,41,51;
        02,12,22,32; 12,22,32,42; 22,32,42,52;
        00,01,02,03; 01,02,03,04; 02,03,04,05; 03,04,05,06;
        10,11,12,13; 11,12,13,14; 12,13,14,15; 13,14,15,16;
        20,21,22,23; 21,22,23,24; 22,23,24,25; 23,24,25,26'''
    # ================================================
    import random
    import numpy as np

    Rows, Cols, weights = 6, 7, [0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125]
    AT2 = (np.array([a for a in range(42)])).reshape(Rows, Cols)
    def yx(s): return [int(s[0]), int(s[1])]
    Ss = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    iMap4q = [list(map(yx, s)) for s in Ss]
    # ================================================
    def agent_NStep(obs, config):
        # valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
        io = it(obs.board, obs.mark)
        tfd_moves = io["moves"]
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        # Use the heuristic to assign a score to each possible board in the next step
        scores = dict(zip(tfd_moves, [score_move4(grid, col, obs.mark, config, N_STEPS) for col in tfd_moves]))
        # Get a list of columns (moves) that maximize the heuristic
        max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
        # Select at random from the maximizing columns
        return random.choice(max_cols)
    # ================================================
    def get_heuristic(grid, mark):
        q = qos(grid, mark)
        score = q[0] * 10000000 + q[1] * 1 - q[2] * 100 - q[3] * 100000
        return score
    # ================================================
    def minimax(node, depth, maximizingPlayer, mark, config):
        # ---------------------------------------------------------------
        if depth == 0 or win_4o_in(node): return get_heuristic(node, mark)
        # ---------------------------------------------------------------
        if mark == 1:
            tfd_moves = iTDF_e(node, mark)["moves"]
        elif mark == 2:
            tfd_1 = iTDF_e(node, mark)
            tfd_2 = iTDF_e(node, 3-mark)
            tfd_moves = tfd_1["moves"] if tfd_1["sum"] > tfd_2["sum"] else tfd_2["moves"]
        # ---------------------------------------------------------------
        if maximizingPlayer:
            value = -np.Inf
            for col in tfd_moves:
                child = drop_piece(node, col, mark, config)
                value = max(value, minimax(child, depth - 1, False, mark, config))
            return value
        else:
            value = np.Inf
            for col in tfd_moves:
                child = drop_piece(node, col, mark % 2 + 1, config)
                value = min(value, minimax(child, depth - 1, True, mark, config))
            return value
    # ================================================
    # Uses minimax to calculate value of dropping piece in selected column
    def score_move4(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        score = minimax(next_grid, nsteps - 1, False, mark, config)
        return score
    # ================================================
    def gpiece(grid, col, nrows=config.rows):
        for row in range(nrows - 1, -1, -1):
            if grid[row][col] == 0: break
        return [row, col]
    # ================================================
    def ipiece(col_move, mark, board, config):
        cboard = np.asarray(board.copy())
        grid = cboard.reshape(config.rows, config.columns)
        yx = gpiece(grid, col_move, nrows=config.rows)
        ia = AT2[yx[0], yx[1]]
        cboard[ia] = mark
        return cboard
    # ================================================
    def gip(col_move, mark, board, config):
        def gpiece2(grid, col, nrows=config.rows):
            for row in range(nrows - 1, -1, -1):
                if grid[row][col] == 0: break
            return [row, col]
        AT2 = (np.array([a for a in range(42)])).reshape(config.rows, config.columns)
        cboard = board.copy()
        grid = np.asarray(cboard).reshape(config.rows, config.columns)
        yx = gpiece2(grid, col_move, nrows=config.rows)
        ia = AT2[yx[0], yx[1]]
        cboard[ia] = mark
        return cboard
    # ================================================
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows - 1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid
    # ================================================
    def win_4o_in(grid):
        for i in iMap4q:
            v = grid[i[0][0], i[0][1]] * grid[i[1][0], i[1][1]] * grid[i[2][0], i[2][1]] * grid[i[3][0], i[3][1]]
            if v == 1 or v == 16: return True
        return False
    # ================================================
    def qos(g, mark):
        gameVecs = [[g[i[0][0],i[0][1]],g[i[1][0],i[1][1]],g[i[2][0],i[2][1]],g[i[3][0],i[3][1]]] for i in iMap4q]
        _mark = 3 - mark
        q4, q3, _q3, _q4 = 0, 0, 0, 0
        for vec in gameVecs:
            _q, q = 0, 0
            if vec[0] == mark: q += 1;
            elif vec[0] == _mark: _q += 1;
            if vec[1] == mark: q += 1;
            elif vec[1] == _mark: _q += 1;
            if vec[2] == mark: q += 1;
            elif vec[2] == _mark: _q += 1;
            if vec[3] == mark:  q += 1;
            elif vec[3] == _mark: _q += 1;
            if q == 3: q3 += 1
            elif q == 4: q4 += 1
            if _q == 3: _q3 += 1
            elif _q == 4: _q4 += 1
        return [q4, q3, _q3, _q4]
    # ================================================
    def bathos2(pie, board, AT2):
        y = pie[0]
        x = pie[1]
        j = AT2[y][x]
        s = -1;
        for i in range(j, len(board), 7):
            if board[i] == 0: s += 1
        return s
    # ================================================
    def ish_4(v, mark, mayak=8):
        if len(v) < 4: return False
        for i in range(0, len(v) - 4 + 1):
            Q = True
            for j in range(0, 4):
                Q &= ((v[i + j][0] == mark) or (v[i + j][0] == mayak))
            if Q:
                return True
        return False
    # ================================================
    def ish_ns_35(v, board, mark, mayak=8):
        for i in range(len(v)):
            x = [e.copy() for e in v]
            if v[i][0] == 0:
                N = x.copy()
                N[i][0] = mark
                ish = ish_4(N, mark)
                if ish:
                    N[i][0] = 4
                    for j in range(len(v)):
                        if N[j][0] == mayak:
                            if bathos2(N[j][1], board, AT2) == 0:
                                N[j][0] = 35
                            else:
                                N[j][0] = 34
                    return [True, N]
        return [False, []]
    # ================================================
    def line(V, a, b):
        A = V[a]["Vector"]
        A.reverse()
        B = V[b]["Vector"][1:]
        return A + B
    # ================================================
    def Total(vector, mark, weights):
        opomark = mark % 2 + 1
        j, ves = 0, 0
        for i in range(1, len(vector)):
            if vector[i][0] == opomark:
                break
            elif vector[i][0] == mark:
                ves += 1
            elif vector[i][0] == 0:
                ves += weights[j]
                j += 1
        return ves
    # ================================================
    def clocs(vpie, grid, mark, weights, AT2, calc_DC=True, Rs=Rows, Cs=Cols):
        mayak = 8
        M = grid
        Y, X = vpie[0], vpie[1]
        V = {}
        Ro = M[Y, X]
        M[Y, X] = mayak
        v = [[M[i][X], [i, X]] for i in range(Y, -1, -1)]
        V["Noth"] = {"Total": Total(v, mark, weights), "Vector": v}
        v = [[M[i][X], [i, X]] for i in range(Y, Rs, 1)]
        V["South"] = {"Total": Total(v, mark, weights), "Vector": v}
        v = [[M[Y][i], [Y, i]] for i in range(X, -1, -1)]
        V["West"] = {"Total": Total(v, mark, weights), "Vector": v}
        v = [[M[Y][i], [Y, i]] for i in range(X, Cs, 1)]
        V["East"] = {"Total": Total(v, mark, weights), "Vector": v}
        v, i, j = [], vpie[0], vpie[1]
        while i >= 0 and j < Cs:
            v.append([M[i][j], [i, j]])
            i, j = i - 1, j + 1
        V["Noth-East"] = {"Total": Total(v, mark, weights), "Vector": v}
        v, i, j = [], vpie[0], vpie[1]
        while i >= 0 and j >= 0:
            v.append([M[i][j], [i, j]])
            i, j = i - 1, j - 1
        V["Noth-West"] = {"Total": Total(v, mark, weights), "Vector": v}
        v, i, j = [], vpie[0], vpie[1]
        while i < Rs and j < Cs:
            v.append([M[i][j], [i, j]])
            i, j = i + 1, j + 1
        V["South-East"] = {"Total": Total(v, mark, weights), "Vector": v}
        v, i, j = [], vpie[0], vpie[1]
        while i < Rs and j >= 0:
            v.append([M[i][j], [i, j]])
            i, j = i + 1, j - 1
        V["South-West"] = {"Total": Total(v, mark, weights), "Vector": v}
        # Noth-West   NOTH   Noth-East
        #             \ | /
        #    WEST    -- o --    EAST
        #             / | \
        # South-West  SOUTH  South-East
        V["Horizont"] = {"Total": V["West"]["Total"] + V["East"]["Total"], "Vector": line(V, "West", "East"), "35": []}
        V["Vertical"] = {"Total": V["Noth"]["Total"] + V["South"]["Total"], "Vector": line(V, "Noth", "South"),"35": []}
        V["Positive"] = {"Total": V["South-West"]["Total"] + V["Noth-East"]["Total"],
                         "Vector": line(V, "South-West", "Noth-East"), "35": []}
        V["Negative"] = {"Total": V["Noth-West"]["Total"] + V["South-East"]["Total"],
                         "Vector": line(V, "Noth-West", "South-East"), "35": []}
        k_H = (len(V["Horizont"]["Vector"]) / 4) ** 2
        k_V = (len(V["Vertical"]["Vector"]) / 4) ** 2
        k_P = (len(V["Positive"]["Vector"]) / 4) ** 2
        k_N = (len(V["Negative"]["Vector"]) / 4) ** 2
        V["Horizont"]["Total"] *= k_H
        V["West"]["Total"] *= k_H
        V["East"]["Total"] *= k_H
        V["Vertical"]["Total"] *= k_V
        V["Noth"]["Total"] *= k_V
        V["South"]["Total"] *= k_V
        V["Positive"]["Total"] *= k_P
        V["South-West"]["Total"] *= k_P
        V["Noth-East"]["Total"] *= k_P
        V["Negative"]["Total"] *= k_N
        V["Noth-West"]["Total"] *= k_N
        V["South-East"]["Total"] *= k_N
        tfd = 0
        tfd += V["Noth"]["Total"]
        tfd += V["Noth-East"]["Total"]
        tfd += V["East"]["Total"]
        tfd += V["South-East"]["Total"]
        tfd += V["South"]["Total"]
        tfd += V["South-West"]["Total"]
        tfd += V["West"]["Total"]
        tfd += V["Noth-West"]["Total"]
        V["TFD"] = {"Total": tfd, "yx": vpie}
        # ///////////////////////////////////////////////////////////////////
        if not calc_DC:
            M[Y, X] = Ro
            return V
        # ///////////////////////////////////////////////////////////////////
        board = np.asarray(grid).reshape(-1)
        V["Horizont"]['35'] = ish_ns_35(V["Horizont"]["Vector"], board, mark)
        V["Vertical"]['35'] = ish_ns_35(V["Vertical"]["Vector"], board, mark)
        V["Positive"]['35'] = ish_ns_35(V["Positive"]["Vector"], board, mark)
        V["Negative"]['35'] = ish_ns_35(V["Negative"]["Vector"], board, mark)
        qdc = 0
        if V["Horizont"]['35'][0]: qdc += 1
        if V["Vertical"]['35'][0]: qdc += 1
        if V["Positive"]['35'][0]: qdc += 1
        if V["Negative"]['35'][0]: qdc += 1
        V["DC"] = {"qnt": qdc, "yx": vpie}
        M[Y, X] = Ro
        return V
    # ================================================
    def iclocs(board, mark, weights, AT2, calc_DC=True, Rs=Rows, Cs=Cols):
        T2 = (np.array(board)).reshape(Rs, Cs)
        vmoves = [c for c in range(Cs) if board[c] == 0]
        vpies = [gpiece(T2, move) for move in vmoves]

        Clocs = [clocs(vpies[i], T2, mark, weights, AT2) for i in range(len(vpies))]

        QntDC, Adrs = 0, []
        SumTfd, queTfd = 0, []
        for cloc in Clocs:
            SumTfd += cloc["TFD"]['Total']
            queTfd.append([cloc["TFD"]['Total'], cloc["TFD"]['yx'][1]])

            if calc_DC == True:

                if cloc["DC"]['qnt'] >= 2:
                    QntDC += 1
                    Adrs.append(cloc["DC"]['yx'])
                    # if cloc["DC"]['qnt'] == 2:   # not here to compare, but you can here but first а в _4_1 == _4_2
                    #     if len(cloc["DC"]['adrs']) >= 2 and cloc["DC"]['adrs'][0] != cloc["DC"]['adrs'][1]:
                    #         QntDC += 1
                    #         Adrs.append(cloc["DC"]['yx'])

        queTfd.sort(reverse=True)

        ItogClocs = {"TFD": {"sum": SumTfd, "que": queTfd}, "DC": {"qnt": QntDC, "adrs": Adrs}}
        Clocs.insert(0, ItogClocs)
        return Clocs
    # ================================================
    def iTDF(clocs, width=WIDTH):
        i = {}
        i["sum"] = clocs[0]["TFD"]['sum']
        i["que"] = [clocs[0]["TFD"]['que'][i] for i in range(len(clocs[0]["TFD"]['que'])) if i < width]
        i["moves"] = [q[1] for q in i["que"]]
        i["qdc"] = clocs[0]["DC"]["qnt"]
        i["idc"] = clocs[0]["DC"]['adrs']
        i["mdc"] = i["idc"][0][1] if i["qdc"] != 0 else []
        # 'sum'    = 36.4375
        # 'que'    = [[8.4375, 5], [8.0, 4], [7.25, 2]]
        # 'moves'  = [5, 4, 2]
        # 'qdc'    = 2
        # 'idc'    = [[3,2], [3,4]]
        # 'mdc'    = 2
        return i
    # ================================================
    def it(board, mark, width=WIDTH, calc_DC=True, Rs=Rows, Cs=Cols):
        clocs = iclocs(board, mark, weights, AT2, calc_DC)
        i = iTDF(clocs, width)
        return i
    # ================================================
    def iTDF_e(grid, mark, weights=weights, AT2=AT2, width=WIDTH):
        vmoves = [c for c in grid[0] if c == 0]
        vpies = [gpiece(grid, move) for move in vmoves]
        clocses = [clocs(vpies[i], grid, mark, weights, AT2) for i in range(len(vpies))]
        SumTfd, queTfd = 0, []
        for cloc in clocses:
            SumTfd += cloc["TFD"]['Total']
            queTfd.append([cloc["TFD"]['Total'], cloc["TFD"]['yx'][1]])
        queTfd.sort(reverse=True)
        ItogClocs = {"TFD": {"sum": SumTfd, "que": queTfd}}
        clocses.insert(0, ItogClocs)
        i = {}
        i["que"] = [clocses[0]["TFD"]['que'][i] for i in range(len(clocses[0]["TFD"]['que'])) if i < width]
        i["sum"]   = clocses[0]["TFD"]['sum'] # 'sum'    = 36.4375
        i["moves"] = [q[1] for q in i["que"]] # 'moves'  = [5, 4, 2]
        return i
    # ================================================
    def deco(X):
        def decomp(x):
            d0, d1, d2, d3 = x[3], x[2], x[1], x[0]
            d4 = [x[3][0], x[2][1], x[1][2], x[0][3]]
            d5 = [x[3][3], x[2][2], x[1][1], x[0][0]]
            d6 = [x[3][0], x[2][0], x[1][0], x[0][0]]
            d7 = [x[3][1], x[2][1], x[1][1], x[0][1]]
            d8 = [x[3][2], x[2][2], x[1][2], x[0][2]]
            d9 = [x[3][3], x[2][3], x[1][3], x[0][3]]
            return [d0, d1, d2, d3, d4, d5, d6, d7, d8, d9]
        x11 = [X[0: 4], X[7:11], X[14:18], X[21:25]]
        x12 = [X[1: 5], X[8:12], X[15:19], X[22:26]]
        x13 = [X[2: 6], X[9:13], X[16:20], X[23:27]]
        x14 = [X[3: 7], X[10:14], X[17:21], X[24:28]]
        x21 = [X[7:11], X[14:18], X[21:25], X[28:32]]
        x22 = [X[8:12], X[15:19], X[22:26], X[29:33]]
        x23 = [X[9:13], X[16:20], X[23:27], X[30:34]]
        x24 = [X[10:14], X[17:21], X[24:28], X[31:35]]
        x31 = [X[14:18], X[21:25], X[28:32], X[35:39]]
        x32 = [X[15:19], X[22:26], X[29:33], X[36:40]]
        x33 = [X[16:20], X[23:27], X[30:34], X[37:41]]
        x34 = [X[17:21], X[24:28], X[31:35], X[38:42]]
        XXX = [[x11, x12, x13, x14], [x21, x22, x23, x24], [x31, x32, x33, x34]]
        Y = []
        for XX in XXX:
            for X in XX: Y.append(decomp(X))
        return Y
    # ================================================
    def f1z(A, m):
        if m == 1:
            if A == [0, 1, 1, 1]: return 0
            if A == [1, 0, 1, 1]: return 1
            if A == [1, 1, 0, 1]: return 2
            if A == [1, 1, 1, 0]: return 3
        if m == 2:
            if A == [0, 2, 2, 2]: return 0
            if A == [2, 0, 2, 2]: return 1
            if A == [2, 2, 0, 2]: return 2
            if A == [2, 2, 2, 0]: return 3
        return -1
    # ================================================
    def f2o(A, m):
        if m == 1:
            if A == [0, 1, 1, 0]: return [0, 3]
        if m == 2:
            if A == [0, 2, 2, 0]: return [0, 3]
        return [-1, -1]
    # ================================================
    def fo0o(A, m):
        if m == 1:
            if A == [0, 1, 0, 1]: return [2]
            if A == [1, 0, 1, 0]: return [1]
        if m == 2:
            if A == [0, 2, 0, 2]: return [2]
            if A == [2, 0, 2, 0]: return [1]
        return [-1]
    # ================================================
    def f2z(A, m):
        if m == 1:
            if A == [1, 1, 0, 0]: return [2, 3]
            if A == [0, 0, 1, 1]: return [0, 1]
            # if A == [1,0,0,1]: return [1,2]
        if m == 2:
            if A == [2, 2, 0, 0]: return [2, 3]
            if A == [0, 0, 2, 2]: return [0, 1]
            # if A == [2,0,0,2]: return [1,2]
        return [-1, -1]
    # ================================================
    def bathos(G, j):
        if j == -1: return -1
        s = -1;
        for i in range(j, len(G), 7):
            if G[i] == 0: s += 1
        return s
    # ================================================
    def shield_1s(G, g, a, m):
        u0, u1, u2 = [], [], []
        for i in range(len(a)):
            for r in range(len(g[i])):
                j = f1z(g[i][r], m)
                u = a[i][r][j]
                if j != - 1:
                    if bathos(G, u) == 0:  u0.append(u)
                    if bathos(G, u) == 1:
                        u1.append(u)
                    else:
                        u2.append(u)

        return [u0, u1, u2]
    # ================================================
    def shield_2s(G, g, a, m):
        T = []
        for i in range(len(a)):
            for r in range(len(g[i])):
                t2 = f2o(g[i][r], m)  # ==============
                j, jj = t2[0], t2[1]
                if (j != -1) and (jj != -1):
                    a1, a2 = a[i][r][j], a[i][r][jj]
                    if (bathos(G, a1) == 0 and bathos(G, a2) == 0):
                        T.append(a1)
                        T.append(a2)
                t1 = fo0o(g[i][r], m)  # ==============
                j = t1[0]
                if (j != -1):
                    a1 = a[i][r][j]
                    if bathos(G, a1) == 0: T.append(a1)
        return T
    # ================================================
    def shield_2z(G, g, a, m):
        Z = []
        for i in range(len(a)):
            for r in range(len(g[i])):
                t = f2z(g[i][r], m)
                j, jj = t[0], t[1]
                if (j != -1) and (jj != -1):
                    a1, a2 = a[i][r][j], a[i][r][jj]
                    if bathos(G, a1) == 0: Z.append(a1)
                    if bathos(G, a2) == 0: Z.append(a2)
        if len(Z) > 0:
            A2d = [[Z.count(s), s] for s in set(Z)]
            A2d.sort()
            E = A2d[-1:][0][1]
            return [E % 7]
        return []
    # ================================================
    def shield_dc(u, io, obs, config):
        idc = [i[1] for i in io['idc'] if i[1] not in u]
        if len(idc) == 1: return idc[0]
        for move in idc:
            obs_board = gip(move, obs.mark, obs.board, config)
            way = deco(obs_board)
            aix = deco([n for n in range(42)])
            I = shield_1s(obs_board, way, aix, obs.mark)
            if len(I[0]) > 0: return I[0][0] % 7
        return random.choice(idc)
    # ================================================
    def select_move(T, U, Z, r, valid_moves):
        u = list(set([u % 7 for u in U[1]]))
        t = list(set([t % 7 for t in T]))
        # -----------
        if len(t) > 0 and (t[0] not in u): return t[0]
        if (r != -1)  and (r not in u):    return valid_moves[r]
        if len(Z) > 0 and (Z[0] not in u): return Z[0]
        # -----------
        for move in valid_moves:
            if move not in u: return move
        # -----------
        return random.choice(valid_moves)
    # =============================================================
    way = deco(obs.board)
    aix = deco([n for n in range(42)])
    I = shield_1s(obs.board, way, aix, obs.mark)
    T = shield_2s(obs.board, way, aix, obs.mark % 2 + 1)
    U = shield_1s(obs.board, way, aix, obs.mark % 2 + 1)
    Z = shield_2z(obs.board, way, aix, obs.mark % 2 + 1)
    Ti= shield_2s(obs.board, way, aix, obs.mark)
    Zi= shield_2z(obs.board, way, aix, obs.mark)
    u = list(map(lambda x: x % 7, U[1]))  # U[1] : o_oo = deep 1
    # ---------------------------------------------------------------------------
    if len(I[0]) > 0: return I[0][0] % 7  # 100% win                # attack  1m
    if len(U[0]) > 0: return U[0][0] % 7  # 100% loss               # defense 1m
    # ---------------------------------------------------------------------------
    io1 = it(obs.board, obs.mark)
    if (io1['qdc'] >= 1 and io1['mdc'] not in u):                   # attack  DC
        return io1['mdc']
    # -----------------------------------------------------
    io2 = it(obs.board, obs.mark % 2 + 1)
    if (io2['qdc'] >= 1):
        if (io2['qdc'] == 1 and io2['mdc'] not in u):               # defense DC
            return io2['mdc']
        elif io2['qdc'] > 1:
            return shield_dc(u, io2, obs, config)
    # ---------------------------------------------------------------------------
    vms = [c for c in range(config.columns) if obs.board[c] == 0]
    _7steps = [[] for c in range(len(vms))]

    for i in range(len(vms)):
        cboard = ipiece(vms[i], obs.mark, obs.board, config)
        io = it(cboard, obs.mark % 2 + 1)
        _7steps[i] = [io['qdc'], io, cboard]
    # ---------------------------------------------------------------------------
    if len(Ti) > 0 and (Ti[0] % 7) not in u: return Ti[0] % 7
    if len(T)  > 0 and (T [0] % 7) not in u: return T [0] % 7
    # ---------------------------------------------------------------------------
    Re, agent_move, i = -1, -1, np.sum(np.array(obs.board))

    if (i > 11): agent_move = agent_NStep(obs, config)

    if (agent_move != -1) and (agent_move not in u): return agent_move

    Re = 3 if i <= 7 else 4 if i <= 11 else -1

    return select_move(T, U, Z, Re, [c for c in range(config.columns) if obs.board[c] == 0])

## [agent-c4_2](https://www.kaggle.com/code/vyacheslavbolotin/agent-c4-2/edit/run/152823182)

In [4]:
def agent_c4_2(obs, config):
    
    WIDTH, N_STEPS = 3, 8
        
    # ================================================
    
    import random
    import numpy as np
    
    def auto_map4q(config,format='str'):
        def add(v, to_lv):
            for vec in to_lv:
                if np.array_equal(v,vec): return to_lv
            to_lv.append(v)
            return to_lv
        grid = np.asarray([[j*10+i for i in range(config.columns)] for j in range(config.rows)])
        lm = []
        for y in range(grid.shape[0] - config.inarow + 1):
            for x in range(grid.shape[1] - config.inarow + 1):
                lm.append(grid[y:y+config.inarow, x:x+config.inarow])
        lv = []
        for m in lm:
            for y in range(config.inarow): lv = add(m[y,:], lv)
            for x in range(config.inarow): lv = add(m[:,x], lv)
            lv = add(m.diagonal(), lv)
            lv = add(np.fliplr(m).diagonal(), lv)
        if format == 'str':
            s = ''
            for v in lv:
                for e in v:
                    _s = str(e)
                    if len(_s) == 1: _s = '0'+ _s
                    s += _s + ','
                s = s.rstrip(",")
                s += '; '
            return s[:-2]
        return lv

    # ================================================
    
    Rows, Cols = config.rows, config.columns
    map4q = auto_map4q(config)
    Vesa = [0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125]
    AT2 = (np.array([a for a in range(42)])).reshape(Rows, Cols)
    def yx(s): return [int(s[0]), int(s[1])]
    Ss = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    iMap4q = [list(map(yx, s)) for s in Ss]
    
    # ====================================================

    def agent_NStep(obs, config):
        # valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
        io = it(obs.board, obs.mark)
        tfd_moves = io["moves"]
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        # Use the heuristic to assign a score to each possible board in the next step
        scores = dict(zip(tfd_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in tfd_moves]))
        # Get a list of columns (moves) that maximize the heuristic
        max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
        # Select at random from the maximizing columns
        return random.choice(max_cols)
    # ================================================
    def get_heuristic(grid, mark):
        q = qos(grid, mark)
        score = 1e7*q[0] + 1e5*q[1] * 1e3+q[2] - 1e2*q[3] - 1e4*q[4] - 1e6*q[5]
        return score
    # ================================================
    def minimax(node, depth, maximizingPlayer, mark, config): # Minimax implementation
        # ---------------------------------------------------------------
        if depth == 0 or win_4o_in(node): return get_heuristic(node, mark)
        # ---------------------------------------------------------------
        tfd_moves = iTDF_e(node, mark)["moves"]
        # ---------------------------------------------------------------
        if maximizingPlayer:
            value = -np.Inf
            for col in tfd_moves:
                child = drop_piece(node, col, mark, config)
                value = max(value, minimax(child, depth - 1, False, mark, config))
            return value
        else:
            value = np.Inf
            for col in tfd_moves:
                child = drop_piece(node, col, mark % 2 + 1, config)
                value = min(value, minimax(child, depth - 1, True, mark, config))
            return value
    # ================================================
    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)
        score = minimax(next_grid, nsteps - 1, False, mark, config)
        return score
    # ================================================
    def gpiece(grid, col, nrows=config.rows):
        for row in range(nrows - 1, -1, -1):
            if grid[row][col] == 0: break
        return [row, col]
    # ================================================
    def ipiece(col_move, mark, board, config):
        cboard = np.asarray(board.copy())
        grid = cboard.reshape(config.rows, config.columns)
        yx = gpiece(grid, col_move, nrows=config.rows)
        ia = AT2[yx[0], yx[1]]
        cboard[ia] = mark
        return cboard
    # ================================================
    def gip(col_move, mark, board, config):
        def gpiece2(grid, col, nrows=config.rows):
            for row in range(nrows - 1, -1, -1):
                if grid[row][col] == 0: break
            return [row, col]
        AT2 = (np.array([a for a in range(42)])).reshape(config.rows, config.columns)
        cboard = board.copy()
        grid = np.asarray(cboard).reshape(config.rows, config.columns)
        yx = gpiece2(grid, col_move, nrows=config.rows)
        ia = AT2[yx[0], yx[1]]
        cboard[ia] = mark
        return cboard
    # ================================================
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows - 1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid
    # ================================================
    def win_4o_in(grid):
        for i in iMap4q:
            v = grid[i[0][0], i[0][1]] * grid[i[1][0], i[1][1]] * grid[i[2][0], i[2][1]] * grid[i[3][0], i[3][1]]
            if v == 1 or v == 16: return True
        return False
    # ================================================
    def qos(g, mark):
        gameVecs = [[g[i[0][0],i[0][1]],g[i[1][0],i[1][1]],g[i[2][0],i[2][1]],g[i[3][0],i[3][1]]] for i in iMap4q]
        _mark = 3 - mark
        q4, q3, q2, _q2, _q3, _q4 = 0, 0, 0, 0, 0, 0
        for vec in gameVecs:
            _q, q = 0, 0
            if vec[0] == mark: q += 1;
            elif vec[0] == _mark: _q += 1;
            if vec[1] == mark: q += 1;
            elif vec[1] == _mark: _q += 1;
            if vec[2] == mark: q += 1;
            elif vec[2] == _mark: _q += 1;
            if vec[3] == mark:  q += 1;
            elif vec[3] == _mark: _q += 1;
            if q == 4: q4 += 1
            elif q == 3: q3 += 1
            elif q == 2: q2 += 1
            if _q == 2: _q2 += 1
            elif _q == 3: _q3 += 1
            elif _q == 4: _q4 += 1
        return [q4, q3, q2, _q2, _q3, _q4]
    # ================================================
    def bathos2(pie, board, AT2):
        y = pie[0]
        x = pie[1]
        j = AT2[y][x]
        s = -1;
        for i in range(j, len(board), 7):
            if board[i] == 0: s += 1
        return s
    # ================================================
    def ish_4(v, mark, mayak=8):
        if len(v) < 4: return False
        for i in range(0, len(v) - 4 + 1):
            Q = True
            for j in range(0, 4):
                Q &= ((v[i + j][0] == mark) or (v[i + j][0] == mayak))
            if Q:
                return True
        return False
    # ================================================
    def ish_ns_35(v, board, mark, mayak=8):
        for i in range(len(v)):
            x = [e.copy() for e in v]
            if v[i][0] == 0:
                N = x.copy()
                N[i][0] = mark
                ish = ish_4(N, mark)
                if ish:
                    N[i][0] = 4
                    for j in range(len(v)):
                        if N[j][0] == mayak:
                            if bathos2(N[j][1], board, AT2) == 0:
                                N[j][0] = 35
                            else:
                                N[j][0] = 34
                    return [True, N]
        return [False, []]
    # ================================================
    def line(V, a, b):
        A = V[a]["Vector"]
        A.reverse()
        B = V[b]["Vector"][1:]
        return A + B
    # ================================================
    def Total(vector, mark, vesa):
        opomark = mark % 2 + 1
        j, ves = 0, 0
        for i in range(1, len(vector)):
            if vector[i][0] == opomark:
                break
            elif vector[i][0] == mark:
                ves += 1
            elif vector[i][0] == 0:
                ves += vesa[j]
                j += 1
        return ves
    # ================================================
    def clocs(vpie, grid, mark, Vesa, AT2, calc_DC=True, Rs=Rows, Cs=Cols):
        mayak = 8
        M = grid
        Y, X = vpie[0], vpie[1]
        V = {}
        Ro = M[Y, X]
        M[Y, X] = mayak
        v = [[M[i][X], [i, X]] for i in range(Y, -1, -1)]
        V["Noth"] = {"Total": Total(v, mark, Vesa), "Vector": v}
        v = [[M[i][X], [i, X]] for i in range(Y, Rs, 1)]
        V["South"] = {"Total": Total(v, mark, Vesa), "Vector": v}
        v = [[M[Y][i], [Y, i]] for i in range(X, -1, -1)]
        V["West"] = {"Total": Total(v, mark, Vesa), "Vector": v}
        v = [[M[Y][i], [Y, i]] for i in range(X, Cs, 1)]
        V["East"] = {"Total": Total(v, mark, Vesa), "Vector": v}
        v, i, j = [], vpie[0], vpie[1]
        while i >= 0 and j < Cs:
            v.append([M[i][j], [i, j]])
            i, j = i - 1, j + 1
        V["Noth-East"] = {"Total": Total(v, mark, Vesa), "Vector": v}
        v, i, j = [], vpie[0], vpie[1]
        while i >= 0 and j >= 0:
            v.append([M[i][j], [i, j]])
            i, j = i - 1, j - 1
        V["Noth-West"] = {"Total": Total(v, mark, Vesa), "Vector": v}
        v, i, j = [], vpie[0], vpie[1]
        while i < Rs and j < Cs:
            v.append([M[i][j], [i, j]])
            i, j = i + 1, j + 1
        V["South-East"] = {"Total": Total(v, mark, Vesa), "Vector": v}
        v, i, j = [], vpie[0], vpie[1]
        while i < Rs and j >= 0:
            v.append([M[i][j], [i, j]])
            i, j = i + 1, j - 1
        V["South-West"] = {"Total": Total(v, mark, Vesa), "Vector": v}
        # Noth-West   NOTH   Noth-East
        #             \ | /
        #    WEST    -- o --    EAST
        #             / | \
        # South-West  SOUTH  South-East
        V["Horizont"] = {"Total": V["West"]["Total"] + V["East"]["Total"], "Vector": line(V, "West", "East"), "35": []}
        V["Vertical"] = {"Total": V["Noth"]["Total"] + V["South"]["Total"], "Vector": line(V, "Noth", "South"),
                         "35": []}
        V["Positive"] = {"Total": V["South-West"]["Total"] + V["Noth-East"]["Total"],
                         "Vector": line(V, "South-West", "Noth-East"), "35": []}
        V["Negative"] = {"Total": V["Noth-West"]["Total"] + V["South-East"]["Total"],
                         "Vector": line(V, "Noth-West", "South-East"), "35": []}
        k_H = (len(V["Horizont"]["Vector"]) / 4) ** 2
        k_V = (len(V["Vertical"]["Vector"]) / 4) ** 2
        k_P = (len(V["Positive"]["Vector"]) / 4) ** 2
        k_N = (len(V["Negative"]["Vector"]) / 4) ** 2
        V["Horizont"]["Total"] *= k_H
        V["West"]["Total"] *= k_H
        V["East"]["Total"] *= k_H
        V["Vertical"]["Total"] *= k_V
        V["Noth"]["Total"] *= k_V
        V["South"]["Total"] *= k_V
        V["Positive"]["Total"] *= k_P
        V["South-West"]["Total"] *= k_P
        V["Noth-East"]["Total"] *= k_P
        V["Negative"]["Total"] *= k_N
        V["Noth-West"]["Total"] *= k_N
        V["South-East"]["Total"] *= k_N
        tfd = 0
        tfd += V["Noth"]["Total"]
        tfd += V["Noth-East"]["Total"]
        tfd += V["East"]["Total"]
        tfd += V["South-East"]["Total"]
        tfd += V["South"]["Total"]
        tfd += V["South-West"]["Total"]
        tfd += V["West"]["Total"]
        tfd += V["Noth-West"]["Total"]
        V["TFD"] = {"Total": tfd, "yx": vpie}
        # ///////////////////////////////////////////////////////////////////
        if not calc_DC:
            M[Y, X] = Ro
            return V
        # ///////////////////////////////////////////////////////////////////
        board = np.asarray(grid).reshape(-1)
        V["Horizont"]['35'] = ish_ns_35(V["Horizont"]["Vector"], board, mark)
        V["Vertical"]['35'] = ish_ns_35(V["Vertical"]["Vector"], board, mark)
        V["Positive"]['35'] = ish_ns_35(V["Positive"]["Vector"], board, mark)
        V["Negative"]['35'] = ish_ns_35(V["Negative"]["Vector"], board, mark)
        qdc = 0
        if V["Horizont"]['35'][0]: qdc += 1
        if V["Vertical"]['35'][0]: qdc += 1
        if V["Positive"]['35'][0]: qdc += 1
        if V["Negative"]['35'][0]: qdc += 1
        V["DC"] = {"qnt": qdc, "yx": vpie}
        M[Y, X] = Ro
        return V
    # ================================================
    def iclocs(board, mark, Vesa, AT2, calc_DC=True, Rs=Rows, Cs=Cols):
        T2 = (np.array(board)).reshape(Rs, Cs)
        vmoves = [c for c in range(Cs) if board[c] == 0]
        vpies = [gpiece(T2, move) for move in vmoves]

        Clocs = [clocs(vpies[i], T2, mark, Vesa, AT2) for i in range(len(vpies))]

        QntDC, Adrs = 0, []
        SumTfd, queTfd = 0, []
        for cloc in Clocs:
            SumTfd += cloc["TFD"]['Total']
            queTfd.append([cloc["TFD"]['Total'], cloc["TFD"]['yx'][1]])

            if calc_DC == True:

                if cloc["DC"]['qnt'] >= 2:
                    QntDC += 1
                    Adrs.append(cloc["DC"]['yx'])
                    # if cloc["DC"]['qnt'] == 2:
                    #     if len(cloc["DC"]['adrs']) >= 2 and cloc["DC"]['adrs'][0] != cloc["DC"]['adrs'][1]:
                    #         QntDC += 1
                    #         Adrs.append(cloc["DC"]['yx'])

        queTfd.sort(reverse=True)

        ItogClocs = {"TFD": {"sum": SumTfd, "que": queTfd}, "DC": {"qnt": QntDC, "adrs": Adrs}}
        Clocs.insert(0, ItogClocs)
        return Clocs
    # ================================================
    def iTDF(clocs, width=WIDTH):
        i = {}
        i["sum"] = clocs[0]["TFD"]['sum']
        i["que"] = [clocs[0]["TFD"]['que'][i] for i in range(len(clocs[0]["TFD"]['que'])) if i < width]
        i["moves"] = [q[1] for q in i["que"]]
        i["qdc"] = clocs[0]["DC"]["qnt"]
        i["idc"] = clocs[0]["DC"]['adrs']
        i["mdc"] = i["idc"][0][1] if i["qdc"] != 0 else []
        # 'sum'    = 36.4375
        # 'que'    = [[8.4375, 5], [8.0, 4], [7.25, 2]]
        # 'moves'  = [5, 4, 2]
        # 'qdc'    = 2
        # 'idc'    = [[3,2], [3,4]]
        # 'mdc'    = 2
        return i
    # ================================================
    def it(board, mark, width=WIDTH, calc_DC=True, Rs=Rows, Cs=Cols):
        clocs = iclocs(board, mark, Vesa, AT2, calc_DC)
        i = iTDF(clocs, width)
        return i
    # ================================================
    def iTDF_e(grid, mark, Vesa=Vesa, AT2=AT2, width=WIDTH):
        vmoves = [c for c in grid[0] if c == 0]
        vpies = [gpiece(grid, move) for move in vmoves]
        clocses = [clocs(vpies[i], grid, mark, Vesa, AT2) for i in range(len(vpies))]
        SumTfd, queTfd = 0, []
        for cloc in clocses:
            SumTfd += cloc["TFD"]['Total']
            queTfd.append([cloc["TFD"]['Total'], cloc["TFD"]['yx'][1]])
        queTfd.sort(reverse=True)
        ItogClocs = {"TFD": {"sum": SumTfd, "que": queTfd}}
        clocses.insert(0, ItogClocs)
        i = {}
        i["que"] = [clocses[0]["TFD"]['que'][i] for i in range(len(clocses[0]["TFD"]['que'])) if i < width]
        i["sum"]   = clocses[0]["TFD"]['sum'] # 'sum'    = 36.4375
        i["moves"] = [q[1] for q in i["que"]] # 'moves'  = [5, 4, 2]
        return i
    # ================================================
    def deco(X):
        def decomp(x):
            d0, d1, d2, d3 = x[3], x[2], x[1], x[0]
            d4 = [x[3][0], x[2][1], x[1][2], x[0][3]]
            d5 = [x[3][3], x[2][2], x[1][1], x[0][0]]
            d6 = [x[3][0], x[2][0], x[1][0], x[0][0]]
            d7 = [x[3][1], x[2][1], x[1][1], x[0][1]]
            d8 = [x[3][2], x[2][2], x[1][2], x[0][2]]
            d9 = [x[3][3], x[2][3], x[1][3], x[0][3]]
            return [d0, d1, d2, d3, d4, d5, d6, d7, d8, d9]
        x11 = [X[0: 4], X[7:11], X[14:18], X[21:25]]
        x12 = [X[1: 5], X[8:12], X[15:19], X[22:26]]
        x13 = [X[2: 6], X[9:13], X[16:20], X[23:27]]
        x14 = [X[3: 7], X[10:14], X[17:21], X[24:28]]
        x21 = [X[7:11], X[14:18], X[21:25], X[28:32]]
        x22 = [X[8:12], X[15:19], X[22:26], X[29:33]]
        x23 = [X[9:13], X[16:20], X[23:27], X[30:34]]
        x24 = [X[10:14], X[17:21], X[24:28], X[31:35]]
        x31 = [X[14:18], X[21:25], X[28:32], X[35:39]]
        x32 = [X[15:19], X[22:26], X[29:33], X[36:40]]
        x33 = [X[16:20], X[23:27], X[30:34], X[37:41]]
        x34 = [X[17:21], X[24:28], X[31:35], X[38:42]]
        XXX = [[x11, x12, x13, x14], [x21, x22, x23, x24], [x31, x32, x33, x34]]
        Y = []
        for XX in XXX:
            for X in XX: Y.append(decomp(X))
        return Y
    # ================================================
    def f1z(A, m):
        if m == 1:
            if A == [0, 1, 1, 1]: return 0
            if A == [1, 0, 1, 1]: return 1
            if A == [1, 1, 0, 1]: return 2
            if A == [1, 1, 1, 0]: return 3
        if m == 2:
            if A == [0, 2, 2, 2]: return 0
            if A == [2, 0, 2, 2]: return 1
            if A == [2, 2, 0, 2]: return 2
            if A == [2, 2, 2, 0]: return 3
        return -1
    # ================================================
    def f2o(A, m):
        if m == 1:
            if A == [0, 1, 1, 0]: return [0, 3]
        if m == 2:
            if A == [0, 2, 2, 0]: return [0, 3]
        return [-1, -1]
    # ================================================
    def fo0o(A, m):
        if m == 1:
            if A == [0, 1, 0, 1]: return [2]
            if A == [1, 0, 1, 0]: return [1]
        if m == 2:
            if A == [0, 2, 0, 2]: return [2]
            if A == [2, 0, 2, 0]: return [1]
        return [-1]
    # ================================================
    def f2z(A, m):
        if m == 1:
            if A == [1, 1, 0, 0]: return [2, 3]
            if A == [0, 0, 1, 1]: return [0, 1]
            # if A == [1,0,0,1]: return [1,2]
        if m == 2:
            if A == [2, 2, 0, 0]: return [2, 3]
            if A == [0, 0, 2, 2]: return [0, 1]
            # if A == [2,0,0,2]: return [1,2]
        return [-1, -1]
    # ================================================
    def bathos(G, j):
        if j == -1: return -1
        s = -1;
        for i in range(j, len(G), 7):
            if G[i] == 0: s += 1
        return s
    # ================================================
    def shield_1s(G, g, a, m):
        u0, u1, u2 = [], [], []
        for i in range(len(a)):
            for r in range(len(g[i])):
                j = f1z(g[i][r], m)
                u = a[i][r][j]
                if j != - 1:
                    if bathos(G, u) == 0:  u0.append(u)
                    if bathos(G, u) == 1:
                        u1.append(u)
                    else:
                        u2.append(u)

        return [u0, u1, u2]
    # ================================================
    def shield_2s(G, g, a, m):
        T = []
        for i in range(len(a)):
            for r in range(len(g[i])):
                t2 = f2o(g[i][r], m)  # ==============
                j, jj = t2[0], t2[1]
                if (j != -1) and (jj != -1):
                    a1, a2 = a[i][r][j], a[i][r][jj]
                    if (bathos(G, a1) == 0 and bathos(G, a2) == 0):
                        T.append(a1)
                        T.append(a2)
                t1 = fo0o(g[i][r], m)  # ==============
                j = t1[0]
                if (j != -1):
                    a1 = a[i][r][j]
                    if bathos(G, a1) == 0: T.append(a1)
        return T
    # ================================================
    def shield_2z(G, g, a, m):
        Z = []
        for i in range(len(a)):
            for r in range(len(g[i])):
                t = f2z(g[i][r], m)
                j, jj = t[0], t[1]
                if (j != -1) and (jj != -1):
                    a1, a2 = a[i][r][j], a[i][r][jj]
                    if bathos(G, a1) == 0: Z.append(a1)
                    if bathos(G, a2) == 0: Z.append(a2)
        if len(Z) > 0:
            A2d = [[Z.count(s), s] for s in set(Z)]
            A2d.sort()
            E = A2d[-1:][0][1]
            return [E % 7]
        return []
    # ================================================
    def shield_dc(u, io, obs, config):
        idc = [i[1] for i in io['idc'] if i[1] not in u]
        if len(idc) == 1: return idc[0]
        for move in idc:
            obs_board = gip(move, obs.mark, obs.board, config)
            way = deco(obs_board)
            aix = deco([n for n in range(42)])
            I = shield_1s(obs_board, way, aix, obs.mark)
            if len(I[0]) > 0: return I[0][0] % 7
        return random.choice(idc)
    # ================================================
    def select_move(T, U, Z, r, valid_moves):
        u = list(set([u % 7 for u in U[1]]))
        t = list(set([t % 7 for t in T]))
        # -----------
        if len(t) > 0 and (t[0] not in u): return t[0]
        if (r != -1) and (r not in u): return valid_moves[r]
        if len(Z) > 0 and (Z[0] not in u): return Z[0]
        # -----------
        for move in valid_moves:
            if move not in u: return move
        # -----------
        return random.choice(valid_moves)
    # ====================================================
    
    way = deco(obs.board)
    aix = deco([n for n in range(42)])
    I = shield_1s(obs.board, way, aix, obs.mark)
    T = shield_2s(obs.board, way, aix, obs.mark % 2 + 1)
    U = shield_1s(obs.board, way, aix, obs.mark % 2 + 1)
    Z = shield_2z(obs.board, way, aix, obs.mark % 2 + 1)
    Ti = shield_2s(obs.board, way, aix, obs.mark)
    Zi = shield_2z(obs.board, way, aix, obs.mark)
    u = list(map(lambda x: x % 7, U[1]))  # U[1] : o_oo = deep 1
    if len(I[0]) > 0: return I[0][0] % 7  # 100% win
    if len(U[0]) > 0: return U[0][0] % 7  # 100% loss
    # =========================================================================== 3.5
    io1 = it(obs.board, obs.mark)
    if (io1['qdc'] >= 1 and io1['mdc'] not in u):                   # attack
        return io1['mdc']                                           # 90% win
    # -----------------------------------------------------
    io2 = it(obs.board, obs.mark % 2 + 1)
    if (io2['qdc'] >= 1):
        if (io2['qdc'] == 1 and io2['mdc'] not in u):               # defense
            return io2['mdc']                                       # 90% loss
        elif io2['qdc'] > 1:
            return shield_dc(u, io2, obs, config)
    # =========================================================================== 3.5
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 2.5
    vms = [c for c in range(config.columns) if obs.board[c] == 0]
    _7steps = [[] for c in range(len(vms))]

    for i in range(len(vms)):
        cboard = ipiece(vms[i], obs.mark, obs.board, config)
        io = it(cboard, obs.mark % 2 + 1)
        _7steps[i] = [io['qdc'], io, cboard]
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 2.5
    # ---------------------------------------------------------------------------
    if len(Ti) > 0 and (Ti[0] % 7) not in u: return Ti[0] % 7  # 80% win
    if len(T)  > 0 and (T[0] % 7) not in u: return T[0] % 7  # 80% loss
    # ---------------------------------------------------------------------------
    Re, agent_move, i = -1, -1, np.sum(np.array(obs.board))

    if (i > 11): agent_move = agent_NStep(obs, config)
    if (agent_move != -1) and (agent_move not in u): return agent_move

    Re = 3 if i <= 7 else 4 if i <= 11 else -1

    return select_move(T, U, Z, Re, [c for c in range(config.columns) if obs.board[c] == 0])

## [agent-c4_3](https://www.kaggle.com/code/vyacheslavbolotin/agent-c4-3/edit/run/155271907)

In [5]:
def agent_c4_3(obs, config):

    WIDTH, N_STEPS = 2, 10

    Add1oppoMove = True  # if True: WIDTH += 1

    # ---------------------------------------------------------
    
    import random
    import numpy as np
    
    def auto_map4q(config,format='str'):
        def add(v, to_lv):
            for vec in to_lv:
                if np.array_equal(v,vec): return to_lv
            to_lv.append(v)
            return to_lv
        grid = np.asarray([[j*10+i for i in range(config.columns)] for j in range(config.rows)])
        lm = []
        for y in range(grid.shape[0] - config.inarow + 1):
            for x in range(grid.shape[1] - config.inarow + 1):
                lm.append(grid[y:y+config.inarow, x:x+config.inarow])
        lv = []
        for m in lm:
            for y in range(config.inarow): lv = add(m[y,:], lv)
            for x in range(config.inarow): lv = add(m[:,x], lv)
            lv = add(m.diagonal(), lv)
            lv = add(np.fliplr(m).diagonal(), lv)
        if format == 'str':
            s = ''
            for v in lv:
                for e in v:
                    _s = str(e)
                    if len(_s) == 1: _s = '0'+ _s
                    s += _s + ','
                s = s.rstrip(",")
                s += '; '
            return s[:-2]
        return lv

    Rows, Cols = config.rows, config.columns
    map4q = auto_map4q(config)
    Vesa = [0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125]
    AT2 = (np.array([a for a in range(42)])).reshape(Rows, Cols)
    def iyx(s): return AT2[int(s[0])][int(s[1])]
    iSs = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    iMapVecs = [list(map(iyx, s)) for s in iSs]

    # -----------------------------------------------------------------------------------------------

    iMapStar = [
        [[0,7,14,21,28,35], [0,1,2,3,4,5,6], [0], [0,8,16,24,32,40]],
        [[1,8,15,22,29,36], [0,1,2,3,4,5,6], [7,1], [1,9,17,25,33,41]],
        [[2,9,16,23,30,37], [0,1,2,3,4,5,6], [14,8,2], [2,10,18,26,34]],
        [[3,10,17,24,31,38], [0,1,2,3,4,5,6], [21,15,9,3], [3,11,19,27]],
        [[4,11,18,25,32,39], [0,1,2,3,4,5,6], [28,22,16,10,4], [4,12,20]],
        [[5,12,19,26,33,40], [0,1,2,3,4,5,6], [35,29,23,17,11,5], [5,13]],
        [[6,13,20,27,34,41], [0,1,2,3,4,5,6], [36,30,24,18,12,6], [6]],
        [[0,7,14,21,28,35], [7,8,9,10,11,12,13], [7,1], [7,15,23,31,39]],
        [[1,8,15,22,29,36], [7,8,9,10,11,12,13], [14,8,2], [0,8,16,24,32,40]],
        [[2,9,16,23,30,37], [7,8,9,10,11,12,13], [21,15,9,3], [1,9,17,25,33,41]],
        [[3,10,17,24,31,38], [7,8,9,10,11,12,13], [28,22,16,10,4], [2,10,18,26,34]],
        [[4,11,18,25,32,39], [7,8,9,10,11,12,13], [35,29,23,17,11,5], [3,11,19,27]],
        [[5,12,19,26,33,40], [7,8,9,10,11,12,13], [36,30,24,18,12,6], [4,12,20]],
        [[6,13,20,27,34,41], [7,8,9,10,11,12,13], [37,31,25,19,13], [5,13]],
        [[0,7,14,21,28,35], [14,15,16,17,18,19,20], [14,8,2], [14,22,30,38]],
        [[1,8,15,22,29,36], [14,15,16,17,18,19,20], [21,15,9,3], [7,15,23,31,39]],
        [[2,9,16,23,30,37], [14,15,16,17,18,19,20], [28,22,16,10,4], [0,8,16,24,32,40]],
        [[3,10,17,24,31,38], [14,15,16,17,18,19,20], [35,29,23,17,11,5], [1,9,17,25,33,41]],
        [[4,11,18,25,32,39], [14,15,16,17,18,19,20], [36,30,24,18,12,6], [2,10,18,26,34]],
        [[5,12,19,26,33,40], [14,15,16,17,18,19,20], [37,31,25,19,13], [3,11,19,27]],
        [[6,13,20,27,34,41], [14,15,16,17,18,19,20], [38,32,26,20], [4,12,20]],
        [[0,7,14,21,28,35], [21,22,23,24,25,26,27], [21,15,9,3], [21,29,37]],
        [[1,8,15,22,29,36], [21,22,23,24,25,26,27], [28,22,16,10,4], [14,22,30,38]],
        [[2,9,16,23,30,37], [21,22,23,24,25,26,27], [35,29,23,17,11,5], [7,15,23,31,39]],
        [[3,10,17,24,31,38], [21,22,23,24,25,26,27], [36,30,24,18,12,6], [0,8,16,24,32,40]],
        [[4,11,18,25,32,39], [21,22,23,24,25,26,27], [37,31,25,19,13], [1,9,17,25,33,41]],
        [[5,12,19,26,33,40], [21,22,23,24,25,26,27], [38,32,26,20], [2,10,18,26,34]],
        [[6,13,20,27,34,41], [21,22,23,24,25,26,27], [39,33,27], [3,11,19,27]],
        [[0,7,14,21,28,35], [28,29,30,31,32,33,34], [28,22,16,10,4], [28,36]],
        [[1,8,15,22,29,36], [28,29,30,31,32,33,34], [35,29,23,17,11,5], [21,29,37]],
        [[2,9,16,23,30,37], [28,29,30,31,32,33,34], [36,30,24,18,12,6], [14,22,30,38]],
        [[3,10,17,24,31,38], [28,29,30,31,32,33,34], [37,31,25,19,13], [7,15,23,31,39]],
        [[4,11,18,25,32,39], [28,29,30,31,32,33,34], [38,32,26,20], [0,8,16,24,32,40]],
        [[5,12,19,26,33,40], [28,29,30,31,32,33,34], [39,33,27], [1,9,17,25,33,41]],
        [[6,13,20,27,34,41], [28,29,30,31,32,33,34], [40,34], [2,10,18,26,34]],
        [[0,7,14,21,28,35], [35,36,37,38,39,40,41], [35,29,23,17,11,5], [35]],
        [[1,8,15,22,29,36], [35,36,37,38,39,40,41], [36,30,24,18,12,6], [28,36]],
        [[2,9,16,23,30,37], [35,36,37,38,39,40,41], [37,31,25,19,13], [21,29,37]],
        [[3,10,17,24,31,38], [35,36,37,38,39,40,41], [38,32,26,20], [14,22,30,38]],
        [[4,11,18,25,32,39], [35,36,37,38,39,40,41], [39,33,27], [7,15,23,31,39]],
        [[5,12,19,26,33,40], [35,36,37,38,39,40,41], [40,34], [0,8,16,24,32,40]],
        [[6,13,20,27,34,41], [35,36,37,38,39,40,41], [41], [1,9,17,25,33,41]]]

    def TMMBorn(obs, config):
        _Tfdmvs = tfd_move(obs.board, obs.mark, iMapStar, Vesa)
        if Add1oppoMove:
            _Tfdmoves_oppo = tfd_move(obs.board, 3-obs.mark, iMapStar, Vesa, width=1)
            if len(_Tfdmoves_oppo) > 0:
                if (_Tfdmoves_oppo[0] not in _Tfdmvs): _Tfdmvs.extend(_Tfdmoves_oppo)
        # Use the heuristic to assign a score to each possible board in the next step
        scores = dict(zip(_Tfdmvs, [score_move(obs.board, col, obs.mark, config, N_STEPS) for col in _Tfdmvs]))
        # Get a list of columns (moves) that maximize the heuristic
        max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
        # Select at random from the maximizing columns
        return random.choice(max_cols)
    # ================================================
    def heuristic(board, mark):
        q = qos(board, mark)
        score = q[0]*10000000 + q[1]*10 - q[2]*100 - q[3]*100000
        return score
    # ================================================

    def minimax(board, depth, a, b, maximizingPlayer, mark, config):
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        if is_oooo_connect(board, mark): return np.inf 
        if is_oooo_connect(board, 3-mark): return -np.inf
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        if depth == 0: return heuristic(board, mark)
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        _Tfdmoves = tfd_move(board, mark, iMapStar, Vesa)
        if Add1oppoMove:
            _Tfdmoves_oppo = tfd_move(board, 3-mark, iMapStar, Vesa, width=1)
            if len(_Tfdmoves_oppo) > 0:
                if (_Tfdmoves_oppo[0] not in _Tfdmoves): _Tfdmoves.extend(_Tfdmoves_oppo)
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        if maximizingPlayer:
            value = -np.Inf
            for col in _Tfdmoves:
                next = inject(mark, board, col, config)
                value = max(value, minimax(next, depth-1, a, b, False, mark, config))
                if value >= b:
                    break
                a = max(a, value)
            return value
        else:
            value = np.Inf
            for col in _Tfdmoves:
                next = inject(3-mark, board, col, config)
                value = min(value, minimax(next, depth-1, a, b, True, mark, config))
                if value <= a:
                    break
                b = min(b, value)
            return value

    # ================================================
    def score_move(board, col, mark, config, nsteps):
        next_board = inject(mark, board, col, config)
        score = minimax(next_board, nsteps-1, -np.Inf, np.Inf, False, mark, config)
        return score
    # ================================================
    def is_oooo_connect(board, mark):
        for i in iMapVecs:
            v = board[i[0]] * board[i[1]] * board[i[2]] * board[i[3]] * mark
            if v == 1 or v == 32: return True
    # ================================================
    def qos(board, mark):
        oppomark = 3-mark
        q4,q3, _q3,_q4 = 0,0, 0,0
        for ivec in iMapVecs:
            q,_q = 0,0
            for iec in ivec:
                if board[iec]==mark: q+=1
                elif board[iec]==oppomark: _q+=1
            if   q==3:  q3+=1
            elif _q==3: _q3+=1
            elif q==4:  q4+=1
            elif _q==4: _q4+=1
        return [q4,q3,_q3,_q4]
    # ================================================ Total freedom density
    def tfd_star(pie, mark, board, mapStar, wts):
        def total(pie, mark, board, avec, wts):
            # avec=[23,24,25,26,27,28,29], board=[..,1,2,0,0,1,1,2,..], pie==(27)
            l,j,t, x, oppomark = -1,0,0, avec.index(pie[1]), 3-mark
            for i in range(x, len(avec), 1):  # ..,(27),28,29]
                value = board[avec[i]]
                if value==oppomark: break
                elif value==mark: t += 1
                elif value==0: t += wts[j]
                l += 1
                j += 1
            j = 0
            for i in range(x, -1, -1):  # [23,24,25,26,(27),..
                value = board[avec[i]]
                if value==oppomark: break
                elif value==mark: t += 1
                elif value==0: t += wts[j]
                l += 1
                j += 1                      # Too simple a form of assessing the surroundings, the rays of a star,
            total = t * ((l / 4) ** 2)    # For a more accurate and high - quality assessment, you need to watch the game
            return total                    # where and which move is “more necessary”, and “translate” them into a given archit.formula
        t = sum([total(pie, mark, board, vec, wts) for vec in mapStar[pie[1]]])
        return [t, pie[0]]
    # ================================================
    def tfd_move(board, mark, mapStar=iMapStar, wts=Vesa, width=WIDTH):
        vpies = [ffaa(board, i) for i in range(Cols) if board[i] == 0]
        tpies = [tfd_star(pie, mark, board, mapStar, wts) for pie in vpies]
        tpies.sort(reverse=True)
        if width > len(tpies): width = len(tpies)
        moves = [tpies[i][1] for i in range(0, width)]
        return moves
    # ================================================
    def ffaa(board, i, config=config):
        ixff = config.columns * (config.rows - 1) + i
        for absolute_address in range(ixff, -1, -7):
            if board[absolute_address] == 0: return [i, absolute_address]
    # ================================================
    def inject(mark, into_board, in_position_col, config):
        i_absolut_address = config.columns * config.rows - config.columns + in_position_col
        for i in range(i_absolut_address, -1, -7):
            if into_board[i] == 0:
                new_board = into_board.copy()
                new_board[i] = mark
                return new_board
        return into_board
    # ================================================
    def fo1(b,mark, a0, a1, a2, a3):
        m = 1 if mark==1 else 2
        if b[a0]==0 and b[a1]==m and b[a2]==m and b[a3]==m: return a0
        if b[a0]==m and b[a1]==0 and b[a2]==m and b[a3]==m: return a1
        if b[a0]==m and b[a1]==m and b[a2]==0 and b[a3]==m: return a2
        if b[a0]==m and b[a1]==m and b[a2]==m and b[a3]==0: return a3
        return -1
    # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
    def fo2(b,mark, a0, a1, a2, a3):
        m = 1 if mark==1 else 2
        if b[a0]==0 and b[a1]==m and b[a2]==0 and b[a3]==m: return [a0,a2]
        if b[a0]==m and b[a1]==0 and b[a2]==m and b[a3]==0: return [a1,a3]
        if b[a0]==m and b[a1]==0 and b[a2]==0 and b[a3]==m: return [a1,a2]
        if b[a0]==m and b[a1]==m and b[a2]==0 and b[a3]==0: return [a2,a3]
        if b[a0]==0 and b[a1]==m and b[a2]==m and b[a3]==0: return [a0,a3]
        if b[a0]==0 and b[a1]==0 and b[a2]==m and b[a3]==m: return [a0,a1]
        return []
    # ================================================
    def bathos(G, j):
        if j == -1: return -1
        s = -1;
        for i in range(j, len(G), 7):
            if G[i] == 0: s += 1
        return s
    # ================================================
    def shield1(avs, board, mark): # G = board [0, 0, 0, 0, .., 0],  a = iMap4q
        u0,u1, = [],[]
        for av in avs:
            i1d = fo1(board, mark, av[0],av[1],av[2],av[3])
            if i1d != - 1:
                d = bathos(board,i1d)
                if d == 0: u0.append(i1d %7)
                elif d == 1: u1.append(i1d %7)
        return u0,u1
    # ================================================
    def Bijo(G,Ai,Bi,Bj):
        bibj = -1
        dBiy,dBjy = bathos(G,Bi),bathos(G,Bj)
        Aix,Bix,Bjx = Ai%7,Bi%7,Bj%7
        if dBiy==0 and 0==dBjy:
            bibj = Aix
        else:
            if   dBiy==0 and dBjy==1 and (Bjx==Aix or Bix==Bjx): bibj = Aix
            elif dBiy==1 and dBjy==0 and (Bix==Aix or Bix==Bjx): bibj = Aix
        return bibj
    # ================================================
    def shield2(avs, board, mark):
        Aro2 = set()
        Aro3,dc,udc = [],[],[]
        for av in avs:
            io = fo1(board, mark, av[0],av[1],av[2],av[3])
            ioo = fo2(board, mark, av[0],av[1],av[2],av[3])
            if io != -1:
                Aro3.append(io)
            if len(ioo) != 0:
                Aro2.add((ioo[0], ioo[1]))
        if len(Aro2) >= 1:
            Aro2 = list(Aro2)
            if len(Aro3) >= 1:
                for j in range(len(Aro3)):
                    Aj = Aro3[j]
                    for i in range(len(Aro2)):
                        Ai,Bi = Aro2[i][0],Aro2[i][1]
                        Aix,Bix,Ajx = Ai%7,Bi%7,Aj%7
                        if Aix==Ajx or Bix==Ajx:
                            dAiy,dBiy,dAjy = bathos(board,Ai),bathos(board,Bi),bathos(board,Aj)
                            if dAiy==0 and dBiy==0 and dAjy==1:
                                if Aix==Ajx:   dc.append(Aix)
                                elif Bix==Ajx: dc.append(Bix)
            if len(Aro2) >= 2:
                for i in range(len(Aro2)):
                    if len(Aro2) >= i +1:
                        A1i,B1i = Aro2[i][0],Aro2[i][1]
                        b = i
                        for j in range(i + 1, len(Aro2)):
                            A2j,B2j = Aro2[j][0],Aro2[j][1]
                            if (A1i==A2j or A1i==B2j) or (B1i==A2j or B1i==B2j):
                                if   (A1i==A2j): Ai,Bi,Aj,Bj = A1i,B1i,A2j,B2j
                                elif (A1i==B2j): Ai,Bi,Aj,Bj = A1i,B1i,B2j,A2j
                                elif (B1i==A2j): Ai,Bi,Aj,Bj = B1i,A1i,A2j,B2j
                                elif (B1i==B2j): Ai,Bi,Aj,Bj = B1i,A1i,B2j,A2j
                                dAiy = bathos(board,Ai)
                                if dAiy==0:
                                    Aix = Bijo(board, Ai, Bi,Bj)
                                    if Aix != -1: dc.append(Aix)
                                elif dAiy==1:
                                    Aix = Bijo(board, Ai, Bi,Bj)
                                    if Aix != -1: udc.append(Aix)
        return dc,udc
    
    # -----------------------------------------------------------------------------------
    s = np.sum(np.array(obs.board))
    if s == 1: return random.choice([0, 1, 2,2, 3,3,3, 4,4, 5])
    if s <  4: return 3
    # -----------------------------------------------------------------------------------   
    v1,z1 = shield1(iMapVecs, obs.board, obs.mark)
    if len(v1) >0: return int(v1[0])                      # win in 1 turn
    # -----------------------------------------------------------------------------------
    o1,u1 = shield1(iMapVecs, obs.board, 3-obs.mark)
    if len(o1) >0: return int(o1[0])                      # victory block in the opponent's 1st turn
    # -----------------------------------------------------------------------------------
    v2,z2 = shield2(iMapVecs, obs.board, obs.mark)
    if len(v2) >0 and v2[0] not in u1: return int(v2[0])  # victory in 2 moves:
    if len(v2) >1 and v2[1] not in u1: return int(v2[1])
    # -----------------------------------------------------------------------------------
    o2,u2 = shield2(iMapVecs, obs.board, 3-obs.mark)
    if len(o2) >0 and o2[0] not in u1: return int(o2[0])  # block victory in 2-turn opponent
    if len(o2) >1 and o2[1] not in u1: return int(o2[1])
    # -----------------------------------------------------------------------------------

    return TMMBorn(obs, config)

## [agent-c4_4](https://www.kaggle.com/code/vyacheslavbolotin/agent-c4-4/edit/run/155274900)

In [6]:
def agent_c4_4(obs, config):

    WIDTH, DELAY = 4, 7.99  # N_STEPS == auto
    
    Add1oppoMove = True     # WIDTH == 4 +1, 

    map4q = '''
               40,41,42,43; 41,42,43,44;       42,43,44,45; 43,44,45,46; 
              50,51,52,53; 51,52,53,54;        52,53,54,55; 53,54,55,56; 
             00,11,22,33; 04,14,24,34;         14,24,34,44; 24,34,44,54; 
            11,22,33,44; 05,15,25,35;          15,25,35,45; 25,35,45,55; 
           00,10,20,30; 06,16,26,36;           16,26,36,46; 26,36,46,56; 
          01,11,21,31; 03,12,21,30;            04,13,22,31; 13,22,31,40;
         02,12,22,32; 05,14,23,32;             14,23,32,41; 23,32,41,50; 
        00,01,02,03; 06,15,24,33; 33,34,35,36; 15,24,33,42; 24,33,42,51; 
        01,02,03,04; 02,03,04,05; 03,04,05,06; 16,25,34,43; 25,34,43,52; 
        10,11,12,13; 11,12,13,14; 12,13,14,15; 26,35,44,53; 20,31,42,53;
        20,21,22,23; 21,22,23,24; 22,23,24,25; 23,33,43,53; 13,23,33,43;
        30,31,32,33; 31,32,33,34; 32,33,34,35; 10,21,32,43; 21,32,43,54; 
                                               22,33,44,55; 01,12,23,34; 
                                               12,23,34,45; 23,34,45,56; 
                                               02,13,24,35; 13,24,35,46; 
                                               03,14,25,36; 03,13,23,33;
                                               10,20,30,40; 20,30,40,50; 
                                               11,21,31,41; 21,31,41,51;
                                               12,22,32,42; 22,32,42,52;
                                               13,14,15,16; 23,24,25,26'''
    import time
    import random
    import numpy as np

    Rows, Cols = 6, 7
    Vesa = [0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125]
    AT2 = (np.array([a for a in range(42)])).reshape(Rows, Cols)
    def iyx(s): return AT2[int(s[0])][int(s[1])]
    iSs = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    iMapVecs = [list(map(iyx, s)) for s in iSs]

    # -----------------------------------------------------------------------------------------------

    iMapStar = [
        [[0,7,14,21,28,35], [0,1,2,3,4,5,6], [0], [0,8,16,24,32,40]],
        [[1,8,15,22,29,36], [0,1,2,3,4,5,6], [7,1], [1,9,17,25,33,41]],
        [[2,9,16,23,30,37], [0,1,2,3,4,5,6], [14,8,2], [2,10,18,26,34]],
        [[3,10,17,24,31,38], [0,1,2,3,4,5,6], [21,15,9,3], [3,11,19,27]],
        [[4,11,18,25,32,39], [0,1,2,3,4,5,6], [28,22,16,10,4], [4,12,20]],
        [[5,12,19,26,33,40], [0,1,2,3,4,5,6], [35,29,23,17,11,5], [5,13]],
        [[6,13,20,27,34,41], [0,1,2,3,4,5,6], [36,30,24,18,12,6], [6]],
        [[0,7,14,21,28,35], [7,8,9,10,11,12,13], [7,1], [7,15,23,31,39]],
        [[1,8,15,22,29,36], [7,8,9,10,11,12,13], [14,8,2], [0,8,16,24,32,40]],
        [[2,9,16,23,30,37], [7,8,9,10,11,12,13], [21,15,9,3], [1,9,17,25,33,41]],
        [[3,10,17,24,31,38], [7,8,9,10,11,12,13], [28,22,16,10,4], [2,10,18,26,34]],
        [[4,11,18,25,32,39], [7,8,9,10,11,12,13], [35,29,23,17,11,5], [3,11,19,27]],
        [[5,12,19,26,33,40], [7,8,9,10,11,12,13], [36,30,24,18,12,6], [4,12,20]],
        [[6,13,20,27,34,41], [7,8,9,10,11,12,13], [37,31,25,19,13], [5,13]],
        [[0,7,14,21,28,35], [14,15,16,17,18,19,20], [14,8,2], [14,22,30,38]],
        [[1,8,15,22,29,36], [14,15,16,17,18,19,20], [21,15,9,3], [7,15,23,31,39]],
        [[2,9,16,23,30,37], [14,15,16,17,18,19,20], [28,22,16,10,4], [0,8,16,24,32,40]],
        [[3,10,17,24,31,38], [14,15,16,17,18,19,20], [35,29,23,17,11,5], [1,9,17,25,33,41]],
        [[4,11,18,25,32,39], [14,15,16,17,18,19,20], [36,30,24,18,12,6], [2,10,18,26,34]],
        [[5,12,19,26,33,40], [14,15,16,17,18,19,20], [37,31,25,19,13], [3,11,19,27]],
        [[6,13,20,27,34,41], [14,15,16,17,18,19,20], [38,32,26,20], [4,12,20]],
        [[0,7,14,21,28,35], [21,22,23,24,25,26,27], [21,15,9,3], [21,29,37]],
        [[1,8,15,22,29,36], [21,22,23,24,25,26,27], [28,22,16,10,4], [14,22,30,38]],
        [[2,9,16,23,30,37], [21,22,23,24,25,26,27], [35,29,23,17,11,5], [7,15,23,31,39]],
        [[3,10,17,24,31,38], [21,22,23,24,25,26,27], [36,30,24,18,12,6], [0,8,16,24,32,40]],
        [[4,11,18,25,32,39], [21,22,23,24,25,26,27], [37,31,25,19,13], [1,9,17,25,33,41]],
        [[5,12,19,26,33,40], [21,22,23,24,25,26,27], [38,32,26,20], [2,10,18,26,34]],
        [[6,13,20,27,34,41], [21,22,23,24,25,26,27], [39,33,27], [3,11,19,27]],
        [[0,7,14,21,28,35], [28,29,30,31,32,33,34], [28,22,16,10,4], [28,36]],
        [[1,8,15,22,29,36], [28,29,30,31,32,33,34], [35,29,23,17,11,5], [21,29,37]],
        [[2,9,16,23,30,37], [28,29,30,31,32,33,34], [36,30,24,18,12,6], [14,22,30,38]],
        [[3,10,17,24,31,38], [28,29,30,31,32,33,34], [37,31,25,19,13], [7,15,23,31,39]],
        [[4,11,18,25,32,39], [28,29,30,31,32,33,34], [38,32,26,20], [0,8,16,24,32,40]],
        [[5,12,19,26,33,40], [28,29,30,31,32,33,34], [39,33,27], [1,9,17,25,33,41]],
        [[6,13,20,27,34,41], [28,29,30,31,32,33,34], [40,34], [2,10,18,26,34]],
        [[0,7,14,21,28,35], [35,36,37,38,39,40,41], [35,29,23,17,11,5], [35]],
        [[1,8,15,22,29,36], [35,36,37,38,39,40,41], [36,30,24,18,12,6], [28,36]],
        [[2,9,16,23,30,37], [35,36,37,38,39,40,41], [37,31,25,19,13], [21,29,37]],
        [[3,10,17,24,31,38], [35,36,37,38,39,40,41], [38,32,26,20], [14,22,30,38]],
        [[4,11,18,25,32,39], [35,36,37,38,39,40,41], [39,33,27], [7,15,23,31,39]],
        [[5,12,19,26,33,40], [35,36,37,38,39,40,41], [40,34], [0,8,16,24,32,40]],
        [[6,13,20,27,34,41], [35,36,37,38,39,40,41], [41], [1,9,17,25,33,41]]]

    # ================================================

    def select_best_of(max_cols):
        if len(max_cols) > 0:
            if 3 in max_cols: return 3
            if 2 in max_cols and 4 in max_cols: return random.choice([2, 4])
            if 2 in max_cols: return 2
            if 4 in max_cols: return 4
            if 1 in max_cols and 5 in max_cols: return random.choice([1, 5])
            if 1 in max_cols: return 1
            if 5 in max_cols: return 5
            if 0 in max_cols and 6 in max_cols: return random.choice([0, 6])
            if 0 in max_cols: return 0
            if 6 in max_cols: return 6
        return random.choice(max_cols)

    # ================================================

    def qos23(board, mark):
        opp_mark = 3 - mark
        q3,q2, _q2,_q3 = 0,0, 0,0
        for ivec in iMapVecs:
            q,_q = 0,0
            for iec in ivec:
                if board[iec] == mark: q += 1
                elif board[iec] == opp_mark: _q += 1
            if    q == 3 and _q == 0:  q3 += 1
            elif _q == 3 and  q == 0: _q3 += 1
            elif  q == 2 and _q == 0:  q2 += 1
            elif _q == 2 and  q == 0: _q2 += 1
        return [q3, q2, _q2, _q3]

    def qos_o(board, mark, o):
        opp_mark = 3-mark
        qo, _qo = 0,0
        for ivec in iMapVecs:
            q, _q = 0,0
            for iec in ivec:
                if board[iec] == mark: q += 1
                elif board[iec] == opp_mark: _q += 1
            if    q==o and _q == 0:  qo += 1
            elif _q==o and  q == 0: _qo += 1
        return [qo, _qo]

    # ================================================

    def heuristic1(board, mark):
        q = qos23(board, mark)
        score = q[0]*1e5 + q[1]*1e3 - q[2]*2 - q[3]*1e4
        return score

    def heuristic2(board, mark):
        q = qos_o(board, mark, 2)
        score = q[0] - q[1]
        return score

    def heuristic3(board, mark):
        q = qos_o(board, mark, 3)
        score = q[0] - q[1]
        return score

    # ================================================

    Add1oppoMove = True  #  -->  WIDTH += 1

    # !!! damage !!! strong !!! EXPERIMENT !!!

    if obs.mark == 1:
        WIDTH = random.choice([2, 3,3])
        heuristic = random.choice([heuristic1,heuristic1,heuristic1,heuristic1, heuristic2,heuristic2, heuristic3])

    if obs.mark == 2:
        WIDTH = random.choice([2,2, 3])
        heuristic = random.choice([heuristic1,heuristic1,heuristic1,heuristic1, heuristic3,heuristic3, heuristic2])

    # ================================================

    def minimax(board, depth, a, b, maximizingPlayer, mark, config):
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        if time.time() > FINISH:           return 0
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        if is_oooo_connect(board, mark):   return np.inf
        if is_oooo_connect(board, 3-mark): return -np.inf
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        if depth == 0:
            key = str(board)
            if key in STATES: return STATES[key]
            h = heuristic(board, mark)
            STATES[key] = h
            return h
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        _Tfdmoves = tfd_move(board, mark, iMapStar, Vesa, width=WIDTH)
        if Add1oppoMove:
            _Tfdmoves_oppo = tfd_move(board, 3 - mark, iMapStar, Vesa, width=1)
            if len(_Tfdmoves_oppo) > 0:
                if (_Tfdmoves_oppo[0] not in _Tfdmoves): _Tfdmoves.extend(_Tfdmoves_oppo)
        # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        if maximizingPlayer:
            value = -np.Inf
            for col in _Tfdmoves:
                next = inject(mark, board, col, config)
                value = max(value, minimax(next, depth - 1, a, b, False, mark, config))
                if value >= b:
                    break
                a = max(a, value)
            return value
        else:
            value = np.Inf
            for col in _Tfdmoves:
                next = inject(3 - mark, board, col, config)
                value = min(value, minimax(next, depth - 1, a, b, True, mark, config))
                if value <= a:
                    break
                b = min(b, value)
            return value

    # ================================================

    def score_move(board, col, mark, config, nsteps):
        next_board = inject(mark, board, col, config)
        score = minimax(next_board, nsteps - 1, -np.Inf, np.Inf, False, mark, config)
        return score

    # ================================================

    def is_oooo_connect(board, mark):
        for i in iMapVecs:
            v = board[i[0]] * board[i[1]] * board[i[2]] * board[i[3]] * mark
            if v == 1 or v == 32: return True

    # ================================================ Total freedom density

    def tfd_star(pie, mark, board, mapStar, wts):
        def total(pie, mark, board, avec, wts):
            # avec=[23,24,25,26,27,28,29], board=[..,1,2,0,0,1,1,2,..], pie==(27)
            l, j, t, x, oppomark = -1, 0, 0, avec.index(pie[1]), 3 - mark
            for i in range(x, len(avec), 1):  # ..,(27),28,29]
                value = board[avec[i]]
                if value == oppomark:
                    break
                elif value == mark:
                    t += 1
                elif value == 0:
                    t += wts[j]
                l += 1
                j += 1
            j = 0
            for i in range(x, -1, -1):  # [23,24,25,26,(27),..
                value = board[avec[i]]
                if value == oppomark:
                    break
                elif value == mark:
                    t += 1
                elif value == 0:
                    t += wts[j]
                l += 1
                j += 1                    # Too simple a form of assessing the surroundings, the rays of a star,
            total = t * ((l / 3.5) ** 2)  # For a more accurate and high - quality assessment, you need to watch the game
            return total                  # where and which move is “more necessary”, and “translate” them into a given archit.formula
        t = sum([total(pie, mark, board, vec, wts) for vec in mapStar[pie[1]]])
        return [t, pie[0]]

    # ================================================

    def tfd_move(board, mark, mapStar, wts, width):
        vpies = [ffaa(board, i) for i in range(Cols) if board[i] == 0]
        tpies = [tfd_star(pie, mark, board, mapStar, wts) for pie in vpies]
        tpies.sort(reverse=True)
        if width > len(tpies): width = len(tpies)
        moves = [tpies[i][1] for i in range(0, width)]
        return moves

    # ================================================

    def ffaa(board, i, config=config):
        ixff = config.columns * (config.rows - 1) + i
        for absolute_address in range(ixff, -1, -7):
            if board[absolute_address] == 0: return [i, absolute_address]

    # ================================================

    def inject(mark, into_board, in_position_col, config):
        i_absolut_address = config.columns * config.rows - config.columns + in_position_col
        for i in range(i_absolut_address, -1, -7):
            if into_board[i] == 0:
                new_board = into_board.copy()
                new_board[i] = mark
                return new_board
        return into_board

    # ================================================

    def fo1(b, mark, a0, a1, a2, a3):
        m = 1 if mark == 1 else 2
        if b[a0] == 0 and b[a1] == m and b[a2] == m and b[a3] == m: return a0
        if b[a0] == m and b[a1] == 0 and b[a2] == m and b[a3] == m: return a1
        if b[a0] == m and b[a1] == m and b[a2] == 0 and b[a3] == m: return a2
        if b[a0] == m and b[a1] == m and b[a2] == m and b[a3] == 0: return a3
        return -1
    # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
    def fo2(b, mark, a0, a1, a2, a3):
        m = 1 if mark == 1 else 2
        if b[a0] == 0 and b[a1] == m and b[a2] == 0 and b[a3] == m: return [a0, a2]
        if b[a0] == m and b[a1] == 0 and b[a2] == m and b[a3] == 0: return [a1, a3]
        if b[a0] == m and b[a1] == 0 and b[a2] == 0 and b[a3] == m: return [a1, a2]
        if b[a0] == m and b[a1] == m and b[a2] == 0 and b[a3] == 0: return [a2, a3]
        if b[a0] == 0 and b[a1] == m and b[a2] == m and b[a3] == 0: return [a0, a3]
        if b[a0] == 0 and b[a1] == 0 and b[a2] == m and b[a3] == m: return [a0, a1]
        return []

    # ================================================

    def bathos(G, j):
        if j == -1: return -1
        s = -1;
        for i in range(j, len(G), 7):
            if G[i] == 0: s += 1
        return s

    # ================================================

    def shield1(avs, board, mark):  # G = board [0, 0, 0, 0, .., 0],  a = iMap4q
        u0, u1, = [], []
        for av in avs:
            i1d = fo1(board, mark, av[0], av[1], av[2], av[3])
            if i1d != - 1:
                d = bathos(board, i1d)
                if d == 0:
                    u0.append(i1d % 7)
                elif d == 1:
                    u1.append(i1d % 7)
        return u0, u1

    # ================================================

    def Bijo(G, Ai, Bi, Bj):
        bibj = -1
        dBiy, dBjy = bathos(G, Bi), bathos(G, Bj)
        Aix, Bix, Bjx = Ai % 7, Bi % 7, Bj % 7
        if dBiy == 0 and 0 == dBjy:
            bibj = Aix
        else:
            if dBiy == 0 and dBjy == 1 and (Bjx == Aix or Bix == Bjx):
                bibj = Aix
            elif dBiy == 1 and dBjy == 0 and (Bix == Aix or Bix == Bjx):
                bibj = Aix
        return bibj

    # ================================================

    def shield2(avs, board, mark):
        Aro2 = set()
        Aro3, dc, udc = [], [], []
        for av in avs:
            io = fo1(board, mark, av[0], av[1], av[2], av[3])
            ioo = fo2(board, mark, av[0], av[1], av[2], av[3])
            if io != -1:
                Aro3.append(io)
            if len(ioo) != 0:
                Aro2.add((ioo[0], ioo[1]))
        if len(Aro2) >= 1:
            Aro2 = list(Aro2)
            if len(Aro3) >= 1:
                for j in range(len(Aro3)):
                    Aj = Aro3[j]
                    for i in range(len(Aro2)):
                        Ai, Bi = Aro2[i][0], Aro2[i][1]
                        Aix, Bix, Ajx = Ai % 7, Bi % 7, Aj % 7
                        if Aix == Ajx or Bix == Ajx:
                            dAiy, dBiy, dAjy = bathos(board, Ai), bathos(board, Bi), bathos(board, Aj)
                            if dAiy == 0 and dBiy == 0 and dAjy == 1:
                                if Aix == Ajx:
                                    dc.append(Aix)
                                elif Bix == Ajx:
                                    dc.append(Bix)
            if len(Aro2) >= 2:
                for i in range(len(Aro2)):
                    if len(Aro2) >= i + 1:
                        A1i, B1i = Aro2[i][0], Aro2[i][1]
                        b = i
                        for j in range(i + 1, len(Aro2)):
                            A2j, B2j = Aro2[j][0], Aro2[j][1]
                            if (A1i == A2j or A1i == B2j) or (B1i == A2j or B1i == B2j):
                                if (A1i == A2j):
                                    Ai, Bi, Aj, Bj = A1i, B1i, A2j, B2j
                                elif (A1i == B2j):
                                    Ai, Bi, Aj, Bj = A1i, B1i, B2j, A2j
                                elif (B1i == A2j):
                                    Ai, Bi, Aj, Bj = B1i, A1i, A2j, B2j
                                elif (B1i == B2j):
                                    Ai, Bi, Aj, Bj = B1i, A1i, B2j, A2j
                                dAiy = bathos(board, Ai)
                                if dAiy == 0:
                                    Aix = Bijo(board, Ai, Bi, Bj)
                                    if Aix != -1: dc.append(Aix)
                                elif dAiy == 1:
                                    Aix = Bijo(board, Ai, Bi, Bj)
                                    if Aix != -1: udc.append(Aix)
        return dc, udc

    # ================================================

    START = time.time()

    s = np.sum(np.array(obs.board))

    if s == 1: return random.choice([0, 1, 2,2, 3, 4,4, 5])
    if s <= 4: return 3
    # -----------------------------------------------------------------------------------
    v1, z1 = shield1(iMapVecs, obs.board, obs.mark)
    if len(v1) > 0: return int(v1[0])                                  # win in 1 turn
    # -----------------------------------------------------------------------------------
    o1, u1 = shield1(iMapVecs, obs.board, 3 - obs.mark)
    if len(o1) > 0: return int(o1[0])                      # victory block in the opponent's 1st turn
    # -----------------------------------------------------------------------------------
    v2, z2 = shield2(iMapVecs, obs.board, obs.mark)
    if len(v2) > 0 and v2[0] not in u1: return int(v2[0])            # victory in 2 moves:
    if len(v2) > 1 and v2[1] not in u1: return int(v2[1])
    # -----------------------------------------------------------------------------------
    o2, u2 = shield2(iMapVecs, obs.board, 3 - obs.mark)
    if len(o2) > 0 and o2[0] not in u1: return int(o2[0])      # block victory in 2-turn opponent
    if len(o2) > 1 and o2[1] not in u1: return int(o2[1])
    # -----------------------------------------------------------------------------------

    STATES = {}

    FINISH = START + DELAY

    N_STEPS, step = 4, 42 - obs.board.count(0)

    _Tfdmvs = tfd_move(obs.board, obs.mark, iMapStar, Vesa, WIDTH)

    if Add1oppoMove:
        _Tfdmvs_oppo = tfd_move(obs.board, 3 - obs.mark, iMapStar, Vesa, width=1)
        if len(_Tfdmvs_oppo) > 0 and (_Tfdmvs_oppo[0] not in _Tfdmvs):
            _Tfdmvs.extend(_Tfdmvs_oppo)

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    scores = dict(zip(_Tfdmvs, [score_move(obs.board, col, obs.mark, config, N_STEPS) for col in _Tfdmvs]))
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

    while N_STEPS < 42 - step:  # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
        N_STEPS += 1
        for vm in _Tfdmvs:
            if scores[vm] == -np.inf: _Tfdmvs.remove(vm)
        if len(_Tfdmvs) == 0: break
        scores_2 = {}
        for col in _Tfdmvs:
            score = score_move(obs.board, col, obs.mark, config, N_STEPS)
            if score == np.inf: return col
            scores_2[col] = score
        if time.time() > FINISH or np.amax(list(scores_2.values())) <= -np.inf:
            break
        scores = scores_2.copy()  # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

    return random.choice(max_cols) #  select_best_of(max_cols)

    # -----------------------------------------------------------------------------------

In [7]:
env__6_7.run([agent_c4_1,"negamax"])
env__6_7.render(mode="ipython", width=500, height=450)

In [8]:
env__6_7.run([agent_c4_2,"negamax"])
env__6_7.render(mode="ipython", width=500, height=450)

In [9]:
env__6_7.run([agent_c4_3,"negamax"])
env__6_7.render(mode="ipython", width=500, height=450)

In [10]:
env__6_7.run([agent_c4_4,"negamax"])
env__6_7.render(mode="ipython", width=500, height=450)

In [11]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(agent_c4_1,"agent_c4_1.py")
write_agent_to_file(agent_c4_2,"agent_c4_2.py")
write_agent_to_file(agent_c4_3,"agent_c4_3.py")
write_agent_to_file(agent_c4_4,"agent_c4_4.py")

<function agent_c4_1 at 0x78a72d5e6560> written to agent_c4_1.py
<function agent_c4_2 at 0x78a72d5e6050> written to agent_c4_2.py
<function agent_c4_3 at 0x78a72d5e7250> written to agent_c4_3.py
<function agent_c4_4 at 0x78a72d5e65f0> written to agent_c4_4.py


## [debugger_c4](https://www.kaggle.com/vyacheslavbolotin/debugger-c4)

In [12]:
def debugger_c4(list_of_agents_to_trace):
    map4q = '''
         01,02,03,04; 31,32,33,34;                 32,33,34,35; 33,34,35,36; 
           40,41,42,43; 41,42,43,44;             42,43,44,45; 43,44,45,46; 
             50,51,52,53; 51,52,53,54;         52,53,54,55; 30,31,32,33;
               04,14,24,34; 14,24,34,44;     24,34,44,54; 05,15,25,35; 
                 06,16,26,36; 03,12,21,30; 15,25,35,45; 25,35,45,55; 
                   16,26,36,46; 26,36,46,56;          23,34,45,56;
                     04,13,22,31; 13,22,31,40;      03,14,25,36; 
                       05,14,23,32; 14,23,32,41;  20,30,40,50; 
                         06,15,24,33; 15,24,33,42;
                           24,33,42,51; 23,32,41,50;
                             03,13,23,33; 13,23,33,43; 
                               16,25,34,43; 25,34,43,52; 
                                 11,22,33,44; 11,21,31,41;
                     01,12,23,34;  02,13,24,35; 12,23,34,45; 
                   00,11,22,33;      26,35,44,53; 23,33,43,53;              
                 02,12,22,32;          21,32,43,54; 22,33,44,55; 
               00,10,20,30; 22,23,24,25; 21,31,41,51; 13,24,35,46;
             01,11,21,31; 10,20,30,40;     12,22,32,42; 22,32,42,52;
           00,01,02,03; 10,21,32,43;         02,03,04,05; 03,04,05,06;
         10,11,12,13; 11,12,13,14;             12,13,14,15; 13,14,15,16;
       20,21,22,23; 21,22,23,24;                 23,24,25,26; 20,31,42,53; 53,54,55,56'''
    
    # -----------------------------------------------------------------------------------------------
    start_games_at_away_first_agent = False # True # => The first agent from the list will start his game either “from home” or “away”,
    only_observation_first_agent    = False # True # => Everyone will play two games with the first: one “at home”, the other “away”, or everyone will play with everyone
    # -----------------------------------------------------------------------------------------------
    class Obs:
        def __init__(self, board, mark):
            self.board = board
            self.mark = mark
    # -----------------------------------------------------------------------------------------------
    class Config:
        def __init__(self, rows, columns, inarow):
            self.rows = rows
            self.columns = columns
            self.inarow = inarow
    # -----------------------------------------------------------------------------------------------
    import time
    import numpy as np
    # -----------------------------------------------------------------------------------------------
    Rows, Cols = 6, 7
    config = Config(Rows, Cols, 4)
    AT2 = (np.array([a for a in range(42)])).reshape(Rows, Cols)
    # -----------------------------------------------------------------------------------------------
    def yx(s): return AT2[int(s[0])][int(s[1])]
    Ss = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    iMap4q = [list(map(yx, s)) for s in Ss]
    # -----------------------------------------------------------------------------------------------
    def gyx(s): return [int(s[0]), int(s[1])]
    gSs = [x.split(",") for x in [x for x in map4q.replace(" ", "").replace("\n", "").split(";")]]
    gMap4q = [list(map(gyx, s)) for s in gSs]
    # -----------------------------------------------------------------------------------------------
    def inject(col, mark, board):
        i_krai = config.columns * (config.rows - 1) + col
        for i in range(i_krai, -1, -7):
            if board[i] == 0:
                new_board = board.copy()
                new_board[i] = mark
                return new_board
        return board
    # -----------------------------------------------------------------------------------------------
    def win_4o_in(board):
        grid = np.asarray(board).reshape(6,7)
        for i in gMap4q:
            v = grid[i[0][0],i[0][1]] * grid[i[1][0],i[1][1]] * grid[i[2][0],i[2][1]] * grid[i[3][0],i[3][1]]
            if v == 1 or v == 16: return True
        return False
    # -----------------------------------------------------------------------------------------------
    def tsum(reagent):
        home = sum(reagent["games"][0]) if len(reagent["games"][0])>0 else 0
        away = sum(reagent["games"][1]) if len(reagent["games"][1])>0 else 0
        return home + away
    # -----------------------------------------------------------------------------------------------
    def ttime(reagent, gt):
        time = reagent["time"][0]  if len(reagent["time"])>0 else 0
        return [round(time+gt, 2)]
    # -----------------------------------------------------------------------------------------------
    def tmoves(reagent, ms):
        moves = reagent["moves"][0] if len(reagent["moves"])>0 else 0
        return [moves+ms]
    # -----------------------------------------------------------------------------------------------
    def tper1move(reagent):
        time = reagent["time"][0]   if len(reagent["time"]) > 0 else 0
        moves = reagent["moves"][0] if len(reagent["moves"])> 0 else 0
        return [round(time / moves, 1)]
    # -----------------------------------------------------------------------------------------------
    def print_to_clip(board, ims, i, attack, defense, stime1, stime2, s1, s2):
        print("1.{0} [{1}]   2.{2} [{3}]".format(name(attack), s1, name(defense), s2),
              "  1.Time =", round(stime1, 2), '  2.Time =', round(stime2, 2), '  q.moves:', i,
              "  1.speed =", round(stime1 / i, 1), "  2.speed =", round(stime2 / i, 1),
              "  \n\nGame:", str(ims).replace(", ", ","),
              "  \n\nBoard:", str(board).replace(", ", ","),"\n")
        bn2d = np.array(board).reshape(Rows, Cols)
        for r in range(len(bn2d)): print(bn2d[r])
        print('{0}{1}'.format('\n',"="*121))
    # -----------------------------------------------------------------------------------------------
    def rec(total_points, _A, _D, ia, id, stime1, stime2):
        reatta = total_points[name(Attack)]
        redefe = total_points[name(Defense)]
        reatta["games"][0].extend([_A])
        redefe["games"][1].extend([_D])
        reatta["T"] = [tsum(reatta)]
        redefe["T"] = [tsum(redefe)]
        reatta["time"] = ttime(reatta, stime1)
        redefe["time"] = ttime(redefe, stime2)
        reatta["moves"] = tmoves(reatta, ia)
        redefe["moves"] = tmoves(redefe, id)
        reatta["speed"] = tper1move(reatta)
        redefe["speed"] = tper1move(redefe)
    # -----------------------------------------------------------------------------------------------
    def agent_work(Agent, board, mark, config):
        obs = Obs(board, mark)
        start = time.time()
        im = Agent(obs, config)
        t = time.time() - start
        board_next = inject(im, obs.mark, board)
        return im, t, board_next
    # -----------------------------------------------------------------------------------------------
    def write_Connect4(ims):
        #     with open("Connect4.txt", "w") as file:
        #         file.write(str(ims))
        pass 
    # -----------------------------------------------------------------------------------------------
    def write_uraConnect(ura_ims, Attack, Defense):
        #     try:
        #         with open("uraConnect.txt", "w") as file:
        #             stf = str(ura_ims) + '\n' + name(Attack) + ' - ' + name(Defense)
        #             file.write(stf)
        #     finally: return
        pass
    # -----------------------------------------------------------------------------------------------
    def name(agent): return agent.__name__.replace("my_","")
    # -----------------------------------------------------------------------------------------------

    Agents = list_of_agents_to_trace # [ag1,ag2,..,agn]

    observatory = Agents[0] if only_observation_first_agent and len(Agents) > 1 else None

    total_points = { name(agent):{"T":[],"games":[[],[]],"time":[],"moves":[],"speed":[]} for agent in Agents }

    ig, board__0 = 1, [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    for home in Agents:
        for away in Agents:
            if home!=away and (observatory==None or observatory!=None and (observatory==home or observatory==away)):
                if ig==1:
                    print("\n")
                    print(ig-1, ":")
                    print('----------------')
                    for itp in total_points.items(): print(itp)
                    ig += 1
                stime1, stime2, imts1, imts2, ims, imt, = 0, 0, [], [], [], [[], []]
                board__1 = board__0
                # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                if start_games_at_away_first_agent:
                    Attack, Defense = away, home
                else:
                    Attack, Defense = home, away
                # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
                total_points_Attack, total_points_Defense = [], []
                print("--------------------------")
                print(Attack.__name__, "  vs  ", Defense.__name__)
                print("-----------------------------------")
                for i in range(1,22):
                    im1, t1, board__2  = agent_work(Attack, board__1, 1, config)
                    stime1 +=t1
                    imt[0] = str(round(t1,0))
                    ims.append(im1)
                    imts1.append(imt[0])
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__2):
                        rec(total_points, 3, 0, i, i-1, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   1 1  1   1    1     1     ', Attack.__name__, "     1     1    1   1  1 1")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip (board__2, ims, i, Attack, Defense, stime1, stime2, 3,0)
                        write_Connect4 (ims)
                        write_uraConnect (ims, Attack, Defense)
                        time.sleep(1)
                        break
                    im2, t2, board__1 = agent_work(Defense, board__2, 2, config)
                    stime2 +=t2
                    imt[1] = str(round(t2, 0))
                    ims.append(im2)
                    imts2.append(imt[1])
                    print('({0}) . {1} > {2}  {3} < {4} . ({5}) --- {6} : {7} '
                          .format(round(t1,1), name(Attack), im1, im2, name(Defense), round(t2,1), i, str(ims).replace(", ",",")))
                    write_uraConnect (ims, Attack, Defense)
                    if win_4o_in(board__1):
                        rec(total_points, 0, 3, i, i, stime1, stime2)
                        print('\nwin in n win in n win in n win in n win in n win in n win in n win in n')
                        print('   2 2  2   2    2     2     ', Defense.__name__,"     2     2    2   2  2 2")
                        print('win in n win in n win in n win in n win in n win in n win in n win in n\n')
                        print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 0,3)
                        write_Connect4 (ims)
                        write_uraConnect(ims, Attack, Defense)
                        time.sleep(1)
                        break
                if not win_4o_in(board__1) and not win_4o_in(board__2):
                    rec(total_points, 1, 2, i, i, stime1, stime2)
                    print('\nDRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W')
                    print('  0 0  0   0    0     ', Attack.__name__, " - ", Defense.__name__, "     0    0   0  0 0")
                    print('DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W DRAW RAW AW W\n')
                    print_to_clip(board__1, ims, i, Attack, Defense, stime1, stime2, 1,2)
                    write_Connect4 (ims)
                    write_uraConnect(ims, Attack, Defense)
                    time.sleep(3)
                print("\n")
                print(ig,":\n") # , ": ", total_points,
                for itp in total_points.items():
                    print(itp)
                print("\n")
                ig +=1

In [13]:
debugger_c4([agent_c4_1, agent_c4_2, agent_c4_3, agent_c4_4]) #  



0 :
----------------
('agent_c4_1', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_c4_2', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_c4_3', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
('agent_c4_4', {'T': [], 'games': [[], []], 'time': [], 'moves': [], 'speed': []})
--------------------------
agent_c4_1   vs   agent_c4_2
-----------------------------------
(0.0) . agent_c4_1 > 3  3 < agent_c4_2 . (0.1) --- 1 : [3,3] 
(0.0) . agent_c4_1 > 3  3 < agent_c4_2 . (0.1) --- 2 : [3,3,3,3] 
(0.0) . agent_c4_1 > 3  3 < agent_c4_2 . (0.0) --- 3 : [3,3,3,3,3,3] 
(0.0) . agent_c4_1 > 5  4 < agent_c4_2 . (0.0) --- 4 : [3,3,3,3,3,3,5,4] 
(0.0) . agent_c4_1 > 4  0 < agent_c4_2 . (7.0) --- 5 : [3,3,3,3,3,3,5,4,4,0] 
(3.7) . agent_c4_1 > 5  5 < agent_c4_2 . (0.0) --- 6 : [3,3,3,3,3,3,5,4,4,0,5,5] 
(3.7) . agent_c4_1 > 1  1 < agent_c4_2 . (6.9) --- 7 : [3,3,3,3,3,3,5,4,4,0,5,5,1,1] 
(3.7) . agent_c4_1 > 1  1 < agent_c